In [ ]:
%load_ext autoreload
%autoreload 2

### Import libraries

In [ ]:
import numpy as np
import pandas as pd
import random
import re
import scipy as sp
import seaborn as sns
import yaml

from copy import deepcopy
from pathlib import Path
from collections import namedtuple, defaultdict

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import rcParams
from statistics import mean
import scienceplots

from assistive_arm.utils.data_preprocessing import read_headers, smooth_dataframe
from assistive_arm.utils.emg_processing import filter_emg, interpolate_dataframe_to_length
from assistive_arm.utils.emg_plotting import plot_every_muscle, plot_muscle_emg
from assistive_arm.utils.optimum_length_utils import get_jacobian, get_rotation_matrix


np.set_printoptions(precision=3, suppress=True)

In [ ]:
# Plot settings
plt.style.use('fivethirtyeight')

rcParams['font.size'] = 12

rcParams['figure.titlesize'] = 16
rcParams['figure.titleweight'] = 'normal'

rcParams['axes.labelsize'] = 14
rcParams['axes.titlesize'] = 16
rcParams['axes.titleweight'] = 'normal'

rcParams['xtick.labelsize'] = 12
rcParams['ytick.labelsize'] = 12

rcParams['axes.grid'] = True
rcParams['grid.linewidth'] = 0.5

# make background white
rcParams['figure.facecolor'] = 'white'
rcParams['axes.facecolor'] = 'white'
rcParams['savefig.facecolor'] = 'white'

rcParams['lines.linewidth'] = 1.5
rcParams['lines.markersize'] = 10

# Box around plot
rcParams['axes.linewidth'] = 0.4
rcParams['axes.edgecolor'] = 'black'

# Add ticks
rcParams['xtick.major.size'] = 5
rcParams['ytick.major.size'] = 5

rcParams['legend.fontsize'] = 12
rcParams['legend.edgecolor'] = 'black'

# Output rcParams as a custom style file

# Load rcParams from a custom style file
# plt.style.use('../paper.mplstyle')

In [ ]:
x = np.linspace(0, 10, 100)
y = np.sin(x)

plt.figure(figsize=(6, 4))
plt.plot(x, y, label='sin(x)')
plt.xlabel('X Axis')
plt.ylabel('Y Axis')
plt.title('Example Plot using SciencePlots')
plt.legend()
plt.show()

In [ ]:
def print_dict_structure(d, indent=0):
    for key, value in d.items():
        print(' ' * indent + f"{key}: {type(value)}")
        if isinstance(value, dict):
            print_dict_structure(value, indent + 4)

In [ ]:
# Load simulation profile
profiles_dir = Path("../torque_profiles")

with open(profiles_dir / "scaled_simulation_profile.csv", "r") as file:
    sim_profile = pd.read_csv(file)

### Load Data

In [ ]:
target_splines = []

for file in sorted((profiles_dir / "spline_profiles").iterdir()):
    if '62' in file.name:
        target_splines.append(pd.read_csv(file))

In [ ]:
subject_dir = Path(f"../subject_logs/")
subjects = sorted([subject for subject in subject_dir.iterdir() if subject.is_dir()])

subject_data = {}

subject_dirs = {}

for subject in subjects:
    subject_name = subject.name
    subject_data[subject_name] = {}
    subject_dirs[subject_name] = {}

    for session in subject.iterdir():
        if session.is_dir():
            session_dir = session
            subject_data[subject_name][session.name] = {}
            subject_dirs[subject_name][session.name] = {}

            with open("../motor_config.yaml", "r") as f:
                motor_config = yaml.load(f, Loader=yaml.FullLoader)

            angle_calibration_path = session_dir / "device_height_calibration.yaml"

            emg_dir = session_dir / "EMG"
            plot_dir = session_dir / "plots"
            plot_dir.mkdir(exist_ok=True)

            subject_dirs[subject_name][session.name]["emg_dir"] = emg_dir
            subject_dirs[subject_name][session.name]["plot_dir"] = plot_dir
            subject_dirs[subject_name][session.name]["angle_calibration_path"] = angle_calibration_path

            # Read EMG file to extract configuration
            emg_file = emg_dir / "assist_0001_emg.tsv" # 2 on treadmill room, 3 on track room
            emg_config = dict()

            relevant_headers = ["FREQUENCY", "CHANNEL_NAMES"]

            for header in read_headers(emg_file, 13, delimiter="\t"):
                if header[0] in relevant_headers:
                    if header[0] == "FREQUENCY":
                        emg_config[header[0]] = float(header[1])
                    else:
                        emg_config[header[0]] = header[1:]
            print("EMG_CONFIG Channel Names:", emg_config["CHANNEL_NAMES"], "\n")
            subject_data[subject_name][session.name]["emg_config"] = emg_config

In [ ]:
for subject in subject_data:
    for session in subject_data[subject]:
        if subject == "MIH01":
            mapping = {
            "Trigno Avanti Duo Mini1_EMG 1": "RF_RIGHT", 
            "Trigno Avanti Duo Mini1_EMG 2": "VM_RIGHT", 
            "Trigno Avanti Duo Mini2_EMG 1": "RF_LEFT",
            "Trigno Avanti Duo Mini2_EMG 2": "VM_LEFT",
        }
        elif subject == "MIH02":
            mapping = {
                "Trigno Avanti Duo Mini1_EMG BLUE": "VM_RIGHT",
                "Trigno Avanti Duo Mini1_EMG YELLOW": "RF_RIGHT",
                "Trigno Avanti Duo Mini2_EMG BLUE": "VM_LEFT",
                "Trigno Avanti Duo Mini2_EMG YELLOW": "RF_LEFT",
                "Trigno Avanti Duo Mini3_EMG BLUE": "BF_RIGHT_1",
                "Trigno Avanti Duo Mini3_EMG YELLOW": "BF_RIGHT_2",
                "Trigno Avanti Duo Mini4_EMG BLUE": "BF_LEFT_1",
                "Trigno Avanti Duo Mini4_EMG YELLOW": "BF_LEFT_2",
            }
        elif subject == "XABI":
            mapping = {
                "Trigno Avanti Duo Mini1_EMG 2": "RF_RIGHT",
                "Trigno Avanti Duo Mini2_EMG 1": "RF_LEFT"
            }

        session_dir = subject_dir / subject / session

        subject_data[subject][session]["emg_config"]["MAPPING"] = mapping

        if not (session_dir / "emg_config.yaml").exists():
            print("Writing EMG config...")
            with open(session_dir / "emg_config.yaml", "w") as f:
                yaml.dump(emg_config, f)
        else:
            print("EMG config already exists")
            # Open existing config
            with open(session_dir / "emg_config.yaml", "r") as f:
                emg_config = yaml.load(f, Loader=yaml.FullLoader)

        with open(angle_calibration_path, 'r') as f:
            angle_calibration = yaml.load(f, Loader=yaml.FullLoader)

        subject_data[subject][session]["angle_calibration"] = angle_calibration

In [ ]:

session_dict = {}
session_dict["MVIC"] = {}
session_dict["MVIC"]["LEFT"] = None
session_dict["MVIC"]["RIGHT"] = None

data_dict = {"EMG": [],
             "MOTOR_DATA": [],
             "MARKER_DATA": []}

session_dict["UNPOWERED"] = {}
session_dict["ASSISTED"] = {}
session_dict["UNPOWERED"]["BEFORE"] = deepcopy(data_dict)
session_dict["UNPOWERED"]["AFTER"] = deepcopy(data_dict)

# Load all session data
profile_to_num = {}
profile_infos = {}

# Skip first 0.5s to avoid initial noise
skip_first = 0.5 #s
remove_unpowered_after = True

# Handle first motor data
for subject in subjects:
    for session in subject.iterdir():
        if not session.is_dir():
            continue
        session_data = deepcopy(session_dict)
        for file_path in sorted(session.iterdir()):
            if file_path.suffix == ".csv":
                if "unpowered" in file_path.stem:
                    df = pd.read_csv(file_path, index_col="time").loc[skip_first:]
                    n = int(file_path.stem.split("_")[1])
                    if n <= 5:
                        session_data["UNPOWERED"]["BEFORE"]["MOTOR_DATA"].append(df)
                    else:
                        remove_unpowered_after = False
                        session_data["UNPOWERED"]["AFTER"]["MOTOR_DATA"].append(df)
                elif "assist" in file_path.stem:
                    df = pd.read_csv(file_path, skiprows=2, index_col="time").loc[skip_first:]

                    profile_info = read_headers(file_path=file_path, rows=2, delimiter=",")
                    time = int(profile_info[0][1])
                    force = int(profile_info[1][1])

                    profile = f"peak_{time}%_{force}N"
                    profile_infos[profile] = {"peak_time": time, "peak_force": force}

                    if profile not in profile_to_num.keys():
                        profile_to_num[profile] = []

                    num = int(file_path.stem.split("_")[-1])
                    profile_to_num[profile].append(num)

                    if profile not in session_data["ASSISTED"].keys():
                        session_data["ASSISTED"][profile] = deepcopy(data_dict)

                    session_data["ASSISTED"][profile]["MOTOR_DATA"].append(df)
        if remove_unpowered_after:
            session_data["UNPOWERED"].pop("AFTER")
        subject_data[subject.name][session.name]["session_data"] = session_data

In [ ]:
for subject in subjects:
    for session in subject.iterdir():
        if not session.is_dir():
            continue
        emg_dir = session / "EMG"
        session_data = subject_data[subject.name][session.name]["session_data"]
        emg_config = subject_data[subject.name][session.name]["emg_config"]
        
        relevant_cols = ["TIME"] + list(emg_config["MAPPING"].keys())
        
        # Handle second EMG data
        for file_path in sorted(emg_dir.iterdir()):
            if file_path.suffix == ".tsv":
                if "unpowered" in file_path.stem:
                    n = int(file_path.stem.split("_")[1])

                    if "marker" in file_path.stem:
                        # Convert mm to meters
                        df = pd.read_csv(file_path, sep="\t", skiprows=11, usecols=["Time", "Hip Left Z", "Hip Right Z"], index_col="Time") / 1000
                        df = df.loc[skip_first:]
                        if n <= 5:
                            session_data["UNPOWERED"]["BEFORE"]["MARKER_DATA"].append(df)
                        else:
                            session_data["UNPOWERED"]["AFTER"]["MARKER_DATA"].append(df)
                    else:
                        df = pd.read_csv(file_path, sep="\t", skiprows=13, usecols=relevant_cols, index_col="TIME")
                        df.rename(columns=emg_config["MAPPING"], inplace=True)
                        df.sort_index(axis=1, inplace=True)
                        df = df.loc[skip_first:]

                        if n <= 5:
                            session_data["UNPOWERED"]["BEFORE"]["EMG"].append(df)
                        else:
                            session_data["UNPOWERED"]["AFTER"]["EMG"].append(df)

                elif "assist" in file_path.stem:
                    if "marker" in file_path.stem:
                        df = pd.read_csv(file_path, sep="\t", skiprows=11, usecols=["Time", "Hip Left Z", "Hip Right Z"], index_col="Time") / 1000
                        df = df.loc[skip_first:]
                        data_type = "MARKER_DATA"
                    else:
                        df = pd.read_csv(file_path, sep="\t", skiprows=13, usecols=relevant_cols, index_col="TIME")
                        df.rename(columns=emg_config["MAPPING"], inplace=True)
                        df.sort_index(axis=1, inplace=True)
                        df = df.loc[skip_first:]
                        data_type = "EMG"
                        
                    num = int(file_path.stem.split("_")[1])

                    for profile, nums in profile_to_num.items():
                        if num in nums:
                            session_data["ASSISTED"][profile][data_type].append(df)
                            
                elif "MVIC" in file_path.stem:
                    df = pd.read_csv(file_path, sep="\t", skiprows=13, usecols=relevant_cols, index_col="TIME")
                    df.rename(columns=emg_config["MAPPING"], inplace=True)
                    
                    side = "LEFT" if "left" in file_path.stem.lower() else "RIGHT"
                    side_prefix = "_" + side

                    if "BF" in file_path.name:  # If it's the BF file
                        bf_col = [col for col in df.columns if "BF" + side_prefix in col]  # Adjust this if your naming convention is different
                        df = df[bf_col]
                    
                    elif "RF" in file_path.name or "VM" in file_path.name:  # If it's the RF or VM file
                        rf_vm_cols = [col for col in df.columns if any(sub in col for sub in ["RF" + side_prefix, "VM" + side_prefix])]  # Adjust this if your naming convention is different
                        df = df[rf_vm_cols]
                    
                    if session_data["MVIC"][side] is not None:
                        # Merge the new data with the existing data
                        session_data["MVIC"][side] = pd.merge(session_data["MVIC"][side], df, left_index=True, right_index=True, how='outer')
                    else:
                        # Initialize with the new data
                        session_data["MVIC"][side] = df

### Drop unnecessary muscles

The idea here is that the recording from some EMG sensors can be noisy, hence we choose the one that shows consistent results with literature.

IMPORTANT: This can only be done after analyzing the data, so make sure to first look at MVIC and corresponding muscle activations

In [ ]:
def get_unique_muscles(session_data):
    unique_muscles = [
        re.sub(pattern=r"(_LEFT|LEFT_|_RIGHT|RIGHT_)", repl="", string=col)  # Remove LEFT/RIGHT variations
        for col in session_data["MVIC"]["LEFT"].columns
    ]
    return unique_muscles

In [ ]:
for subject in subjects:
    for session in subject.iterdir():
        if not session.is_dir():
            continue
        session_data = subject_data[subject.name][session.name]["session_data"]
        unique_muscles = get_unique_muscles(session_data)
        print(f"Subject: {subject.name}, Session: {session.name}, Unique Muscles: {unique_muscles}")

In [ ]:
session_data = subject_data["MIH02"]["December_20"]["session_data"]

to_drop = ["BF_LEFT_2", "BF_RIGHT_1"] # For MIH02

for side in session_data["MVIC"].keys():
    emg_df = session_data["MVIC"][side]
    emg_df.drop(columns=to_drop, inplace=True, errors="ignore")
    col_mapping = {col: "_".join(col.split("_")[:2]) for col in emg_df.columns}

    emg_df.rename(columns=col_mapping, inplace=True)

for t_unpowered in session_data["UNPOWERED"].keys():
    for emg_df in session_data["UNPOWERED"][t_unpowered]["EMG"]:
        emg_df.drop(columns=to_drop, inplace=True, errors="ignore")
        col_mapping = {col: "_".join(col.split("_")[:2]) for col in emg_df.columns}
        emg_df.rename(columns=col_mapping, inplace=True)

for profile in session_data["ASSISTED"].keys():
    for data_type in session_data["ASSISTED"][profile].keys():
        if data_type != "EMG":
            continue
        for emg_df in session_data["ASSISTED"][profile][data_type]:
            emg_df.drop(columns=to_drop, inplace=True, errors="ignore")
            col_mapping = {col: "_".join(col.split("_")[:2]) for col in emg_df.columns}
            emg_df.rename(columns=col_mapping, inplace=True)

unique_muscles = get_unique_muscles(session_data)

### Remove incomplete data (if any)

Sometimes motor data collection may go wrong and result in empty dataframes or durations that are significantly shorter in time than EMG or MARKER data

In [ ]:
# Track which dataframe is removed for being empty (profile, datatype and index) and for what reason
removed_dfs = {}

In [ ]:
for subject in subjects:
    for session in subject.iterdir():
        if not session.is_dir():
            continue
        session_data = subject_data[subject.name][session.name]["session_data"]
        for profile in session_data["ASSISTED"].keys():
            non_empty_dfs = [(emg_df, motor_df, marker_df) for emg_df, motor_df, marker_df in zip(*session_data["ASSISTED"][profile].values()) if not emg_df.empty and not motor_df.empty and not marker_df.empty]
            for i, (emg_df, motor_df, marker_df) in enumerate(zip(*session_data["ASSISTED"][profile].values())):
                emg_df.dropna(inplace=True)
                motor_df.dropna(inplace=True)
                marker_df.dropna(inplace=True)
                
                if emg_df.empty:
                    removed_dfs[(profile, "EMG", i)] = "empty"
                    session_data["ASSISTED"][profile]["EMG"][i] = non_empty_dfs[0][0].copy()
                    session_data["ASSISTED"][profile]["EMG"][i].loc[:] = 1

                if motor_df.empty:
                    removed_dfs[(profile, "MOTOR_DATA", i)] = "empty"
                    session_data["ASSISTED"][profile]["MOTOR_DATA"][i] = non_empty_dfs[0][1].copy()
                    session_data["ASSISTED"][profile]["MOTOR_DATA"][i].loc[:] = 1

                if marker_df.empty:
                    removed_dfs[(profile, "MARKER_DATA", i)] = "empty"
                    session_data["ASSISTED"][profile]["MARKER_DATA"][i] = non_empty_dfs[0][2].copy()
                    session_data["ASSISTED"][profile]["MARKER_DATA"][i].loc[:] = 1


                # if emg_df.empty or motor_df.empty or marker_df.empty:
                #     # Fill data with random index (will be deleted later, just for consistency)
                #     fill_df = session_data["ASSISTED"][profile]["EMG"][2].copy()
                #     session_data["ASSISTED"][profile]["MOTOR_DATA"][i].loc[:] = 1
                #     session_data["ASSISTED"][profile]["MARKER_DATA"][i].loc[:] = 1
        print(subject / session)
        print(removed_dfs)

### Filter all EMG data

In [ ]:
for subject in subjects:
    for session in subject.iterdir():
        if not session.is_dir():
            continue
        session_data = subject_data[subject.name][session.name]["session_data"]
        subject_data[subject.name][session.name]["filtered_session_data"] = deepcopy(session_data)
        
        filtered_session_data = subject_data[subject.name][session.name]["filtered_session_data"]

        filtered_session_data["MVIC"]["LEFT"], _ = filter_emg(session_data["MVIC"]["LEFT"], sfreq=emg_config["FREQUENCY"])
        filtered_session_data["MVIC"]["RIGHT"], _ = filter_emg(session_data["MVIC"]["RIGHT"], sfreq=emg_config["FREQUENCY"])

        for profile, emg_data in session_data["ASSISTED"].items():
            for i, emg_df in enumerate(session_data["ASSISTED"][profile]["EMG"]):
                filtered_session_data["ASSISTED"][profile]["EMG"][i], _ = filter_emg(emg_df, sfreq=emg_config["FREQUENCY"])

        for unpow_key in session_data["UNPOWERED"].keys():
            for i, emg_df in enumerate(session_data["UNPOWERED"][unpow_key]["EMG"]):
                filtered_session_data["UNPOWERED"][unpow_key]["EMG"][i], _ = filter_emg(emg_df, sfreq=emg_config["FREQUENCY"])

### Plot MAX Activation data

In [ ]:

for subject in subject_data.keys():
    for session in subject_data[subject].keys():    
        filtered_session_data = subject_data[subject][session]["filtered_session_data"]

        n_cols = filtered_session_data["MVIC"]["LEFT"].shape[1]

        fig, axs = plt.subplots(2, n_cols, figsize=(5*n_cols, 5))
        fig.suptitle(f"Subject {subject}\nMVIC for various muscles")

        for i, side in enumerate(filtered_session_data["MVIC"].keys()):
            for j, col in enumerate(filtered_session_data["MVIC"][side].columns):
                if n_cols == 1:
                    axs[i].plot(filtered_session_data["MVIC"][side].index, filtered_session_data["MVIC"][side][col])
                    axs[i].set_title(f"{col}")
                    axs[i].set_xlabel("Time (s)")
                    axs[i].set_ylabel("EMG signal (mV)")
                else:
                    axs[i, j].plot(filtered_session_data["MVIC"][side].index, filtered_session_data["MVIC"][side][col])
                    axs[i, j].set_title(f"{col}")
                    axs[i, j].set_xlabel("Time (s)")
                    axs[i, j].set_ylabel("EMG signal (mV)")
        plt.tight_layout()
        plt.savefig(subject_dirs[subject][session]['plot_dir'] / "MVIC.png", dpi=300, bbox_inches='tight')

In [ ]:
subject = "MIH02"
session = "December_20"

session_data = subject_data[subject][session]["session_data"]
filtered_session_data = subject_data[subject][session]["filtered_session_data"]

unique_muscles = get_unique_muscles(session_data)

for muscle in unique_muscles:
    for profile in session_data["ASSISTED"].keys():
        plot_dir = subject_dirs[subject][session]["plot_dir"]
        filename = plot_dir / f"EMG_{muscle}_{profile}.png"
        if not filename.exists():
            plot_muscle_emg(title=f"EMG data (unscaled)\n{profile}", 
                    target_muscle=muscle,
                    unfiltered_dfs=session_data["ASSISTED"][profile]["EMG"],
                    filtered_dfs=filtered_session_data["ASSISTED"][profile]["EMG"], 
                    freq=emg_config["FREQUENCY"],
                    fig_path=filename,
                    show=False)
        else:
            print(f"Figure {filename} already exists, skipping...")





### Scale data based on MVIC

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        filtered_session_data = subject_data[subject][session]["filtered_session_data"]
        left_mvic_cols = list(filtered_session_data["MVIC"]["LEFT"].columns)
        right_mvic_cols = list(filtered_session_data["MVIC"]["RIGHT"].columns)

        max_left = list(filtered_session_data["MVIC"]["LEFT"].max())
        max_right = list(filtered_session_data["MVIC"]["RIGHT"].max())

        # Create dataframe containing the max values for each muscle, should be 1 row
        mvic_df = pd.DataFrame(data=[max_left + max_right], columns=left_mvic_cols + right_mvic_cols)
        mvic_df = mvic_df[filtered_session_data["UNPOWERED"]["BEFORE"]["EMG"][0].columns]
        
        for t_unpowered in filtered_session_data["UNPOWERED"].keys():
            for i, unpowered_df in enumerate(filtered_session_data["UNPOWERED"][t_unpowered]["EMG"]):
                unpowered_df /= mvic_df.iloc[0]

        for profile in filtered_session_data["ASSISTED"].keys():
            for i, filtered_emg in enumerate(filtered_session_data["ASSISTED"][profile]["EMG"]):
                filtered_emg /= mvic_df.iloc[0]

#### Scale EMG based on reported peak activation (based on papers)

In [ ]:
subject = "MIH02"
session = "December_20"
profile = "peak_77%_62N"

unique_muscles = get_unique_muscles(subject_data[subject][session]["filtered_session_data"])

plot_every_muscle(title=f"Powered EMGs (filtered)\n{profile}", muscles=unique_muscles, dfs=subject_data[subject][session]["filtered_session_data"]["ASSISTED"][profile]["EMG"])

In [ ]:
subject = "MIH02"
session = "December_20"
profile = "peak_77%_62N"

t_unpowered = "BEFORE"

filtered_session_data = subject_data[subject][session]["filtered_session_data"]
unique_muscles = get_unique_muscles(subject_data[subject][session]["filtered_session_data"])

plot_every_muscle(title=f"Unpowered EMG data\n{t_unpowered}", muscles=unique_muscles, dfs=filtered_session_data["UNPOWERED"][t_unpowered]["EMG"][:5])

In [ ]:
subject = "MIH02"
session = "December_20"
profile = "peak_77%_62N"

filtered_session_data = subject_data[subject][session]["filtered_session_data"]
session_data = subject_data[subject][session]["session_data"]

unique_muscles = get_unique_muscles(subject_data[subject][session]["filtered_session_data"])
for muscle in unique_muscles:
    for profile in session_data["ASSISTED"].keys():
        plot_dir = subject_dirs[subject][session]["plot_dir"]
        filename = plot_dir / f"EMG_scaled_{muscle}_{profile}.png"
        if not filename.exists():
            plot_muscle_emg(title=f"EMG data (unscaled)\n{profile}", 
                    target_muscle=muscle,
                    unfiltered_dfs=session_data["ASSISTED"][profile]["EMG"],
                    filtered_dfs=filtered_session_data["ASSISTED"][profile]["EMG"],
                    fig_path=filename,
                    show=False)
        else:
            print(f"Figure {filename} already exists, skipping...")

In [ ]:
plot_muscle_emg(title=f"Powered EMGs (unfiltered VS filtered)\n{profile}",
                target_muscle="RF",
         unfiltered_dfs=session_data["ASSISTED"][profile]["EMG"],
         filtered_dfs=filtered_session_data["ASSISTED"][profile]["EMG"])

## Calculate force generated by motors

In [ ]:
subject = "MIH02"
session = "December_20"
profile = "peak_77%_62N"

motor_log = subject_data[subject][session]["filtered_session_data"]["ASSISTED"][profile]["MOTOR_DATA"][0]

colors = iter(rcParams['axes.prop_cycle'].by_key()['color'])

fig, ax = plt.subplots(3, 1, sharex=True, figsize=(10, 5))
fig.suptitle(profile)

ax[0].plot(motor_log.index, motor_log["theta_2"], label="theta_2", color=colors.__next__())
# ax[0].plot(motor_log.index, angle_calibration["theta_2_values"], label="theta_2 at calibration")
ax[0].axhline(y=angle_calibration["new_range"]["min"], linestyle="--", color="black")
ax[0].axhline(y=angle_calibration["new_range"]["max"], linestyle="--", color="black")
handles0, labels0 = ax[0].get_legend_handles_labels()
ax[0].set_ylabel("theta_2 (rad)")


ax[1].plot(motor_log.index, motor_log["target_tau_1"], label="target_tau_1", color=colors.__next__())
ax[1].plot(motor_log.index, motor_log["measured_tau_1"], label="measured_tau_1", color=colors.__next__())
ax[1].plot(motor_log.index, motor_log["target_tau_2"], label="target_tau_2", color=colors.__next__())
ax[1].plot(motor_log.index, motor_log["measured_tau_2"], label="measured_tau_2", color=colors.__next__())
handles1, labels1 = ax[1].get_legend_handles_labels()
ax[1].set_ylabel("Torque (Nm)")

ax[2].plot(motor_log.index, motor_log["Percentage"], label="STS %", color=colors.__next__())
ax[2].axhline(y=100, linestyle="--", color="black")
ax[2].axhline(y=0, linestyle="--", color="black")
handles2, labels2 = ax[2].get_legend_handles_labels()
ax[2].set_ylabel("STS %")
ax[2].set_xlabel('time(s)')

handles = handles0 + handles1 + handles2
labels = labels0 + labels1 + labels2
fig.legend(handles, labels, loc='upper right')
plt.tight_layout()
plt.show()

### Fix wrap-around from motor measurements

In [ ]:
# Control variable to toggle the sign
torque_limits = [motor_config[motor]["T_max"] for motor in motor_config.keys()]

for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        filtered_session_data = subject_data[subject][session]["filtered_session_data"]

        for profile in filtered_session_data["ASSISTED"].keys():
            motor_data = filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"]

            measured_torques = [column for column in motor_data[0].columns if "measured_tau" in column]

            for motor_df in motor_data:
                for torque, jump_threshold in zip(measured_torques, torque_limits):
                    differences = motor_df[torque].diff().fillna(0)

                    # Threshold for detecting a jump
                    toggle_sign = 1

                    corrected_values = []

                    for diff in differences:
                        if abs(diff) > jump_threshold:
                            toggle_sign *= -1
                        corrected_values.append(toggle_sign)
                    motor_df[torque] *= corrected_values

In [ ]:
subject = "MIH02"
session = "December_20"
profile = "peak_77%_62N"

motor_log = subject_data[subject][session]["filtered_session_data"]["ASSISTED"][profile]["MOTOR_DATA"][0]

fig, ax = plt.subplots(3, 1, sharex=True, figsize=(10, 5))
fig.suptitle(profile)

ax[0].plot(motor_log.index, motor_log["theta_2"], label="theta_2")
# ax[0].plot(motor_log.index, angle_calibration["theta_2_values"], label="theta_2 at calibration")
ax[0].axhline(y=angle_calibration["new_range"]["min"], linestyle="--", color="black")
ax[0].axhline(y=angle_calibration["new_range"]["max"], linestyle="--", color="black")
ax[0].legend()
ax[0].set_ylabel("theta_2 (rad)")


ax[1].plot(motor_log.index, motor_log["target_tau_1"], label="target_tau_1")
ax[1].plot(motor_log.index, motor_log["measured_tau_1"], label="measured_tau_1")
ax[1].plot(motor_log.index, motor_log["target_tau_2"], label="target_tau_2")
ax[1].plot(motor_log.index, motor_log["measured_tau_2"], label="measured_tau_2")
ax[1].legend()
ax[1].set_ylabel("Torque (Nm)")

ax[2].plot(motor_log.index, motor_log["Percentage"], label="STS %")
ax[2].axhline(y=100, linestyle="--", color="black")
ax[2].axhline(y=0, linestyle="--", color="black")
ax[2].legend()
ax[2].set_ylabel("STS %")
ax[2].set_xlabel('time(s)')

### Calculate force from torques

In [ ]:
rotate_90 = get_rotation_matrix(90)

for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        filtered_session_data = subject_data[subject][session]["filtered_session_data"]
                
        for profile in filtered_session_data["ASSISTED"].keys():
            for motor_log in filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"]:
                measured_torques = motor_log[["measured_tau_1", "measured_tau_2"]].to_numpy()
                measured_torques = measured_torques.reshape(*measured_torques.shape, 1)

                jacobians = get_jacobian(l1=0.44, l2=0.41, theta_1=motor_log["theta_1"].to_numpy(), theta_2=motor_log["theta_2"].to_numpy())
                F = - np.linalg.inv(jacobians.T) @ measured_torques
                # F = - rotate_90[:2, :2] @ np.linalg.inv(jacobians.T) @ torques

                motor_log["F_X"] = F[:, 0]
                motor_log["F_Y"] = F[:, 1]
                

In [ ]:
subject = "MIH02"
session = "December_20"
profile = "peak_77%_62N"

filtered_session_data = subject_data[subject][session]["filtered_session_data"]

n_cols = len(filtered_session_data["ASSISTED"].keys())
n_rows = len(filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"])

target_muscle = "RF_LEFT"

fig, axs = plt.subplots(n_rows, n_cols, figsize=(20, 10))
for i, profile in enumerate(filtered_session_data["ASSISTED"].keys()):
    axs[0, i].set_title(f"{profile}\nIteration {1}")
    for j, motor_df in enumerate(filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"]):
        emg_data = filtered_session_data["ASSISTED"][profile]["EMG"][j]

        # Plot force values on the primary y-axis
        axs[j, i].plot(motor_df.index, motor_df.F_Y, label="F_Y", color='orange')
        axs[j, i].set_ylabel('Force (N)', color='orange')
        axs[j, i].tick_params(axis='y', labelcolor='orange')

        # Create a secondary y-axis for EMG data
        ax2 = axs[j, i].twinx()
        ax2.plot(emg_data.index, emg_data[target_muscle], label=target_muscle, color=u'#1f77b4')
        ax2.set_ylim(0, 1)  # Set EMG activation scale from 0 to 1
        ax2.set_ylabel('EMG (% MVIC)', color=u'#1f77b4')
        ax2.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
        ax2.tick_params(axis='y', labelcolor=u'#1f77b4')

        if i == 0:
            axs[j, 0].set_ylabel("Force (N)")
        if j != 0:
            axs[j, i].set_title(f"Iteration {j+1}")
        axs[-1, i].set_xlabel("Time (s)")

plt.tight_layout()
plt.show()

### Remove faulty data

In [ ]:
test = filtered_session_data["ASSISTED"]["peak_77%_62N"]["MOTOR_DATA"][2]
plt.plot(test.Percentage, test.F_Y, label="F_Y")
plt.plot(test.Percentage, test.target_tau_1, label="target_tau_1")
plt.plot(test.Percentage, test.measured_tau_1, label="measured_tau_1")
plt.plot(test.Percentage, test.target_tau_2, label="target_tau_1")
plt.plot(test.Percentage, test.measured_tau_2, label="measured_tau_2")
plt.plot(test.Percentage, test.theta_1, label="theta_1")
plt.plot(test.Percentage, test.theta_2, label="theta_2")
plt.legend()

### Calculate relevant ranges for UNPOWERED and ASSISTED

In [ ]:
time_range = namedtuple("TimeRange", ["min", "max"])

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        filtered_session_data = subject_data[subject][session]["filtered_session_data"]              

        relevant_ranges = {}

        for profile in filtered_session_data["ASSISTED"].keys():
            # relevant_ranges[profile] = []
            relevant_ranges[profile] = {"ranges": [], "hip_vel": []}

            for marker_data in filtered_session_data["ASSISTED"][profile]["MARKER_DATA"]:
                hip_vel = smooth_dataframe(marker_data["Hip Left Z"].diff() / marker_data.index.diff(), window_size=30)

                baseline = hip_vel.loc[:1]
                baseline_avg = baseline.mean() + 0.05 # add offset to be safe

                condition = hip_vel > baseline_avg

                first_index = condition.idxmax() - 0.2
                last_index = condition.iloc[::-1].idxmax() + 0.7

                relevant_ranges[profile]["ranges"].append(time_range(first_index, last_index))
                relevant_ranges[profile]["hip_vel"].append(hip_vel)

        for unpowered_time in filtered_session_data["UNPOWERED"].keys():
            relevant_ranges[f"UNPOWERED_{unpowered_time}"] = {"ranges": [], "hip_vel": []}

            for marker_data in filtered_session_data["UNPOWERED"][unpowered_time]["MARKER_DATA"]:
                hip_vel = smooth_dataframe(marker_data["Hip Left Z"].diff() / marker_data.index.diff(), window_size=30)

                baseline = hip_vel.loc[:1]
                baseline_avg = baseline.mean() + 0.05

                condition = hip_vel > baseline_avg

                first_index = condition.idxmax() - 0.2
                last_index = condition.iloc[::-1].idxmax() + 0.7

                relevant_ranges[f"UNPOWERED_{unpowered_time}"]["ranges"].append(time_range(first_index, last_index))
                relevant_ranges[f"UNPOWERED_{unpowered_time}"]["hip_vel"].append(hip_vel)
        
        subject_data[subject][session]["relevant_ranges"] = relevant_ranges

In [ ]:
# Extract durations from the time ranges
fig, ax = plt.subplots(1, ncols=len(subject_data.keys()), figsize=(10, 5))
fig.suptitle("Durations per profile and subject")

for i, subject in enumerate(subject_data.keys()):
    for session in subject_data[subject].keys():
        relevant_ranges = subject_data[subject][session]["relevant_ranges"]

        durations = {profile: [time_range.max - time_range.min for time_range in time_ranges["ranges"]] for profile, time_ranges in relevant_ranges.items()}

        # Drop empty keys
        durations = {k: v for k, v in durations.items() if v}
        durations_df = pd.DataFrame.from_dict(durations, orient='index').T

        # Plot mean and std of the durations
        durations_df.mean().plot(kind='bar', yerr=durations_df.std(), ax=ax[i])
        ax[i].set_ylabel("Duration (s)")
        ax[i].set_xlabel("Profile")
        ax[i].set_title(f"Subject {subject}")
        ax[i].set_xticklabels(ax[i].get_xticklabels(), rotation=45, ha='right')
        plt.tight_layout()
        # plt.show()


In [ ]:
for profile in relevant_ranges.keys():
    print(profile, relevant_ranges[profile]["ranges"])

In [ ]:
subject = "MIH02"
session = "December_20"
profile = "peak_77%_62N"

filtered_session_data = subject_data[subject][session]["filtered_session_data"]
unique_muscles = get_unique_muscles(subject_data[subject][session]["filtered_session_data"])

profiles = filtered_session_data["ASSISTED"].keys()
n_rows = len(filtered_session_data["ASSISTED"][profile]["MARKER_DATA"])

fig, axs = plt.subplots(n_rows, len(profiles), figsize=(7 * n_rows, 3* n_rows), sharey=True)  # Adjust as needed
fig.suptitle(f"Subject {subject}\nSession {session}\nHip height and speed for each profile")

for col, profile in enumerate(profiles):
    for row in range(n_rows):
        relevant_ranges = subject_data[subject][session]["relevant_ranges"]

        marker_df = filtered_session_data["ASSISTED"][profile]["MARKER_DATA"][row]  # Adjusted for demonstration
        cur_range = relevant_ranges[profile]["ranges"][row]  # Assuming relevant_ranges is structured similarly
        hip_vel = relevant_ranges[profile]["hip_vel"][row]  # Assuming relevant_ranges is structured similarly

        axs[row, col].plot(marker_df.index, marker_df["Hip Left Z"], label="Height", color=u'#1f77b4')
        axs[row, col].tick_params(axis='y', labelcolor=u'#1f77b4')
        axs[row, col].axvline(cur_range.min, color="red", linestyle="--")#, label='Min Range')
        axs[row, col].axvline(cur_range.max, color="red", linestyle="--")#, label='Max Range')  # Using blue for differentiation

        ax2 = axs[row, col].twinx()
        ax2.plot(hip_vel.index, hip_vel, label="Speed", color="orange")
        ax2.tick_params(axis='y', labelcolor="orange")
        if col == len(profiles) - 1:
            ax2.set_ylabel("Speed (m/s)")
        ax2.set_ylim(0, 1)

        handles1, labels1 = axs[row, col].get_legend_handles_labels()
        handles2, labels2 = ax2.get_legend_handles_labels()
        handles = handles1 + handles2
        labels = labels1 + labels2


        axs[0, col].set_title(f"{profile}")
        axs[row, 0].set_ylabel(f"Iteration {row+1}\nHeight (m)")
        axs[-1, col].set_xlabel("Time (s)")

        
fig.legend(handles, labels, loc="upper right", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
subject = "MIH02"
session = "December_20"
profile = "peak_77%_62N"

filtered_session_data = subject_data[subject][session]["filtered_session_data"]
unique_muscles = get_unique_muscles(subject_data[subject][session]["filtered_session_data"])
relevant_ranges = subject_data[subject][session]["relevant_ranges"]

fig, axs = plt.subplots(nrows=3, ncols=5, sharex=True, figsize=(20, 5))
fig.suptitle(f"Unpowered activations (BEFORE trials)\n{subject} - {profile}")

t_unpowered = "BEFORE"

target_muscle = random.choice(unique_muscles)

for i, t_range in enumerate(relevant_ranges[f"UNPOWERED_{t_unpowered}"]["ranges"]):
    marker_data = filtered_session_data["UNPOWERED"][t_unpowered]["MARKER_DATA"][i] 
    motor_data = filtered_session_data["UNPOWERED"][t_unpowered]["MOTOR_DATA"][i]
    emg_data = filtered_session_data["UNPOWERED"][t_unpowered]["EMG"][i]

    axs[0, i].set_title(f"unpowered_{i}")
    axs[0, i].plot(marker_data.index, marker_data["Hip Left Z"])
    axs[0, i].set_ylabel("Hip Z (m)")
    axs[0, i].axvline(t_range.min, color="red")
    axs[0, i].axvline(t_range.max, color="red")
    axs[0, i].grid(True)

    axs[1, i].plot(motor_data.index, motor_data["theta_2"])
    axs[1, i].set_ylabel("theta_2 (rad)")
    axs[1, i].axvline(t_range.min, color="red")
    axs[1, i].axvline(t_range.max, color="red")
    axs[1, i].grid(True)

    axs[2, i].plot(emg_data.index, emg_data[f"{target_muscle}_LEFT"], label=f"{target_muscle}_LEFT")
    axs[2, i].plot(emg_data.index, emg_data[f"{target_muscle}_RIGHT"], label=f"{target_muscle}_RIGHT")
    axs[2, i].set_xlabel("Time (s)")
    axs[2, i].set_ylabel("EMG (% of MVIC)")
    axs[2, i].legend()
    axs[2, i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
    axs[2, i].axvline(t_range.min, color="red")
    axs[2, i].axvline(t_range.max, color="red")
    axs[2, i].grid(True)
plt.tight_layout()

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        filtered_session_data = subject_data[subject][session]["filtered_session_data"]

        for profile in filtered_session_data["ASSISTED"].keys():
            for i in range(len(filtered_session_data["ASSISTED"][profile]["MARKER_DATA"])):
                marker_data = filtered_session_data["ASSISTED"][profile]["MARKER_DATA"][i] 
                motor_data = filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"][i]
                emg_data = filtered_session_data["ASSISTED"][profile]["EMG"][i]

                t_diff = emg_data.index[-1] - motor_data.index[-1]
                if t_diff > 0.5:
                    motor_data.index += t_diff

In [ ]:
subject = "MIH02"
session = "December_20"
profile = "peak_37%_62N"

filtered_session_data = subject_data[subject][session]["filtered_session_data"]

n_rows = len(filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"])
n_cols = len(filtered_session_data["ASSISTED"].keys())

target_muscle = "RF"

fig, axs = plt.subplots(n_rows, n_cols, figsize=(20, 10))
fig.suptitle(f"Subject {subject}\nSession {session}\nForce and EMG for each profile")

for i, profile in enumerate(filtered_session_data["ASSISTED"].keys()):
    axs[0, i].set_title(f"{profile}\nIteration {1}")
    
    for j, motor_df in enumerate(filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"]):
        emg_data = filtered_session_data["ASSISTED"][profile]["EMG"][j]

        # Plot force values on the primary y-axis
        axs[j, i].plot(motor_df.index, motor_df.F_Y, label="F_Y", color='orange')
        axs[j, i].yaxis.set_major_formatter(mticker.StrMethodFormatter("{x:.0f} N"))
        axs[j, i].tick_params(axis='y', labelcolor='orange')
        axs[j, i].grid(True)

        # Create a secondary y-axis for EMG data
        ax2 = axs[j, i].twinx()
        ax2.plot(emg_data.index, emg_data[f"{target_muscle}_RIGHT"], label="RIGHT", color='green')
        ax2.plot(emg_data.index, emg_data[f"{target_muscle}_LEFT"], label="LEFT", color=u'#1f77b4')
        ax2.set_ylim(0, 0.5)  # Set EMG activation scale from 0 to 1
        if i == len(profiles) - 1:
            ax2.set_ylabel('EMG (% MVIC)', color=u'#1f77b4')
        ax2.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
        ax2.tick_params(axis='y', labelcolor=u'#1f77b4')

        handles1, labels1 = axs[j, i].get_legend_handles_labels()
        handles2, labels2 = ax2.get_legend_handles_labels()
        handles = handles1 + handles2
        labels = labels1 + labels2

        axs[j, 0].set_ylabel(f"Force (N)\n{target_muscle}")

        if j != 0:
            axs[j, i].set_title(f"Iteration {j+1}")
        axs[-1, i].set_xlabel("Time (s)")

fig.legend(handles, labels, loc="upper right", fontsize=13)
plt.tight_layout()
plt.show()

## Choose data that you want to keep

In [ ]:
print(len(subject_data["MIH02"]["December_20"]["relevant_ranges"]["peak_37%_62N"]["ranges"]))
print(len(subject_data["MIH02"]["December_20"]["filtered_session_data"]["ASSISTED"]["peak_37%_62N"]["MOTOR_DATA"]))

### Delete specific iterations

In [ ]:
subject_data.pop("XABI", None)

for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        filtered_session_data = subject_data[subject][session]["filtered_session_data"] 
        relevant_ranges = subject_data[subject][session]["relevant_ranges"]

        data_to_keep = {key: None for key in filtered_session_data["ASSISTED"].keys()}

        if subject == "MIH01":
            data_to_keep['peak_27%_62N'] = [0, 1, 2]
            data_to_keep['peak_37%_62N'] = [0, 2, 3]
            data_to_keep['peak_47%_62N'] = [0, 2, 3]
            data_to_keep['peak_57%_62N'] = [1, 2, 3]
            data_to_keep['peak_67%_62N'] = [2, 3, 4]
            data_to_keep['peak_77%_62N'] = [1, 2, 3]

        elif subject == "MIH02":
            data_to_keep['peak_27%_62N'] = [0, 1, 2]
            data_to_keep['peak_37%_62N'] = [1, 2, 3]
            data_to_keep['peak_47%_62N'] = [0, 2, 3]
            data_to_keep['peak_57%_62N'] = [0, 2, 3]
            data_to_keep['peak_67%_62N'] = [0, 1, 3]
            data_to_keep['peak_77%_62N'] = [0, 1, 2]

        elif subject == "XABI":
            data_to_keep['peak_27%_62N'] = [0, 1, 2]
            data_to_keep['peak_37%_62N'] = [0, 1, 3]
            data_to_keep['peak_47%_62N'] = [0, 2, 3]
            data_to_keep['peak_57%_62N'] = [0, 2, 3]
            data_to_keep['peak_67%_62N'] = [0, 1, 3]
            data_to_keep['peak_77%_62N'] = [3]

        for profile in filtered_session_data["ASSISTED"].keys():
            filtered_session_data["ASSISTED"][profile]["EMG"] = [filtered_session_data["ASSISTED"][profile]["EMG"][i] for i in data_to_keep[profile]]
            filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"] = [filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"][i] for i in data_to_keep[profile]]
            filtered_session_data["ASSISTED"][profile]["MARKER_DATA"] = [filtered_session_data["ASSISTED"][profile]["MARKER_DATA"][i] for i in data_to_keep[profile]]

            # Update ranges as well
            relevant_ranges[profile]["ranges"] = [relevant_ranges[profile]["ranges"][i] for i in data_to_keep[profile]]
            relevant_ranges[profile]["hip_vel"] = [relevant_ranges[profile]["hip_vel"][i] for i in data_to_keep[profile]]

            assert len(filtered_session_data["ASSISTED"][profile]["EMG"]) == len(filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"]) == len(filtered_session_data["ASSISTED"][profile]["MARKER_DATA"])

### Delete specific muscles

In [ ]:
muscles_to_remove = ["VM_LEFT", "VM_RIGHT"]

if False:
    for subject in subject_data.keys():
        for session in subject_data[subject].keys():
            filtered_session_data = subject_data[subject][session]["filtered_session_data"]

            for profile in filtered_session_data["ASSISTED"].keys():
                for emg_df in filtered_session_data["ASSISTED"][profile]["EMG"]:
                    emg_df.drop(columns=muscles_to_remove, inplace=True, errors="ignore")
            for t_unpowered in filtered_session_data["UNPOWERED"].keys():
                for emg_df in filtered_session_data["UNPOWERED"][t_unpowered]["EMG"]:
                    emg_df.drop(columns=muscles_to_remove, inplace=True, errors="ignore")
            for side in filtered_session_data["MVIC"].values():
                side.drop(columns=muscles_to_remove, inplace=True, errors="ignore")

In [ ]:
n_prof = len([profile for profile in relevant_ranges.keys() if 'unpowered' not in profile.lower()])

fig, axs = plt.subplots(nrows=4, ncols=n_prof, sharex=True, figsize=(20, 5))

for i, (profile, range_list) in enumerate(relevant_ranges.items()):
    if profile.startswith("UNPOWERED"):
        continue
    df_index = 2

    rel_range = range_list["ranges"][df_index] # Get first iteration range

    marker_data = filtered_session_data["ASSISTED"][profile]["MARKER_DATA"][df_index] 
    motor_data = filtered_session_data["ASSISTED"][profile]["MOTOR_DATA"][df_index]
    emg_data = filtered_session_data["ASSISTED"][profile]["EMG"][df_index]
    
    axs[0, i].set_title(profile)
    axs[0, i].plot(marker_data.index, marker_data["Hip Left Z"])
    axs[0, i].set_ylabel("Hip Z (m)")
    axs[0, i].axvline(rel_range.min, color="red")
    axs[0, i].axvline(rel_range.max, color="red")
    axs[0, i].grid(True)

    axs[1, i].plot(motor_data.index, motor_data["theta_2"])
    axs[1, i].set_ylabel(r"$\theta_2$ (rad)")
    axs[1, i].axvline(rel_range.min, color="red")
    axs[1, i].axvline(rel_range.max, color="red")
    axs[1, i].grid(True)

    ax2 = axs[1, i].twinx()
    ax2.set_ylim(0, 100)
    ax2.plot(motor_data.index, motor_data.Percentage, label="STS %", color='g', linestyle='--')
    ax2.set_ylabel('STS %', color='g')
    ax2.tick_params(axis='y', labelcolor='g')

    axs[2, i].plot(motor_data.index, motor_data["F_Y"])
    axs[2, i].set_ylabel("F_Y (N)")
    axs[2, i].axvline(rel_range.min, color="red")
    axs[2, i].axvline(rel_range.max, color="red")
    axs[2, i].grid(True)

    axs[3, i].plot(emg_data.index, emg_data[f"{target_muscle}_LEFT"], label=f"{target_muscle}_LEFT")
    axs[3, i].plot(emg_data.index, emg_data[f"{target_muscle}_RIGHT"], label=f"{target_muscle}_RIGHT")
    axs[3, i].legend(fontsize=8)

    axs[3, i].set_xlabel("Time (s)")
    axs[3, i].set_ylabel("EMG\n(% of MVIC)")
    axs[3, i].axvline(rel_range.min, color="red")
    axs[3, i].axvline(rel_range.max, color="red")
    axs[3, i].grid(True)
    axs[3, i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))

plt.tight_layout()

In [ ]:
profile = "peak_67%_62N"
ranges = relevant_ranges[profile]["ranges"]

fig, axs = plt.subplots(len(ranges), 3, figsize=(15, 5))
fig.suptitle(f"Profile {profile}")

for i, (emg_df, motor_df, marker_df) in enumerate(zip(*filtered_session_data["ASSISTED"][profile].values())):
    cur_range = ranges[i]

    if i == 0:
        axs[i, 0].set_title(f"Iteration {i+1} (MARKER_DATA)")
        axs[i, 1].set_title(f"Iteration {i+1} (MOTOR_DATA)")
        axs[i, 2].set_title(f"Iteration {i+1} (EMG_DATA)")
    else:
        axs[i, 0].set_title(f"Iteration {i+1}")
        axs[i, 1].set_title(f"Iteration {i+1}")
        axs[i, 2].set_title(f"Iteration {i+1}")

    axs[i, 0].plot(marker_df.index, marker_df["Hip Left Z"], label="Hip Z")
    axs[i, 0].axvline(cur_range.min, color="red")
    axs[i, 0].axvline(cur_range.max, color="red")
    axs[i, 0].set_ylabel("Hip Z (m)")

    hip_vel = relevant_ranges[profile]['hip_vel'][i]
    ax2 = axs[i, 0].twinx() 
    ax2.plot(hip_vel.index, hip_vel, label="Hip Vel", color='orange')
    ax2.set_ylabel("Hip Vel (m/s)")
    ax2.tick_params(axis='y', labelcolor='orange')
    ax2.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.1f'))

    axs[i, 1].plot(motor_df.index, motor_df["F_Y"], label=r"$F_Y$", color='purple')
    axs[i, 1].set_ylabel(r"$F_Y (N)$")
    axs[i, 1].axvline(cur_range.min, color="red")
    axs[i, 1].axvline(cur_range.max, color="red")
    
    ax2 = axs[i, 1].twinx()
    ax2.plot(motor_df.index, motor_df["Percentage"], label="STS %", color='g', linestyle='--')
    ax2.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=100))
    ax2.set_ylim(0, 100)

    axs[i, 2].plot(emg_df.index, emg_df["RF_LEFT"], label=r"$RF_{left}$")
    axs[i, 2].plot(emg_df.index, emg_df["RF_RIGHT"], label=r"$RF_{right}$")
    axs[i, 2].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
    axs[i, 2].axvline(cur_range.min, label=r"$t_0$", color="red")
    axs[i, 2].axvline(cur_range.max, label=r"$t_f$", color="red")
    axs[i, 2].set_ylabel("EMG\n(% of MVIC)")

    handles0, labels0 = axs[i, 0].get_legend_handles_labels()
    handles1, labels1 = axs[i, 1].get_legend_handles_labels()
    handles2, labels2 = axs[i, 2].get_legend_handles_labels()
    handles3, labels3 = ax2.get_legend_handles_labels()

    handles = handles0 + handles1 + handles2 + handles3
    labels = labels0 + labels1 + labels2 + labels3


    


fig.legend(handles, labels, loc="upper center", ncol=len(labels), bbox_to_anchor=(0.5, 0.95))
plt.tight_layout(rect=(0, 0, 1, 0.95))
plt.show()

## Mean baseline activations for every muscle

### Cut off relevant time ranges based on activation

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        filtered_session_data = subject_data[subject][session]["filtered_session_data"]
        relevant_ranges = subject_data[subject][session]["relevant_ranges"]

        cut_off_dfs = {"ASSISTED": dict(), "UNPOWERED": dict()}

        for profile in filtered_session_data["ASSISTED"].keys():
            rel_ranges = relevant_ranges[profile]["ranges"] # Get first iteration range
            cut_off_dfs["ASSISTED"][profile] = deepcopy(data_dict)

            emg_dfs, motor_dfs, marker_dfs = filtered_session_data["ASSISTED"][profile].values()


            cut_off_dfs["ASSISTED"][profile]["EMG"] = [emg_df.loc[rel_range.min:rel_range.max] for emg_df, rel_range in zip(emg_dfs, rel_ranges)]
            cut_off_dfs["ASSISTED"][profile]["MOTOR_DATA"] = [motor_df.loc[rel_range.min:rel_range.max] for motor_df, rel_range in zip(motor_dfs, rel_ranges)]
            cut_off_dfs["ASSISTED"][profile]["MARKER_DATA"] = [marker_df.loc[rel_range.min:rel_range.max] for marker_df, rel_range in zip(marker_dfs, rel_ranges)]

            for motor_df, rel_range in zip(motor_dfs, rel_ranges):
                if rel_range.min > motor_df.index[-1]:
                    print("Warning: cut-off range starts after motor data. Keeping whole range")
                    rel_range = rel_range._replace(min=motor_df.index[0])
                # cut_off_dfs["ASSISTED"][profile]["MOTOR_DATA"].append(motor_df.loc[rel_range.min:rel_range.max])

        for t_unpowered in filtered_session_data["UNPOWERED"].keys():
            rel_ranges = relevant_ranges[f"UNPOWERED_{t_unpowered}"]["ranges"] # Get first iteration range

            cut_off_dfs["UNPOWERED"][t_unpowered] = deepcopy(data_dict)

            emg_dfs, motor_dfs, marker_dfs = filtered_session_data["UNPOWERED"][t_unpowered].values()

            cut_off_dfs["UNPOWERED"][t_unpowered]["EMG"] = [emg_df.loc[rel_range.min:rel_range.max] for emg_df, rel_range in zip(emg_dfs, rel_ranges)]
            cut_off_dfs["UNPOWERED"][t_unpowered]["MARKER_DATA"] = [marker_df.loc[rel_range.min:rel_range.max] for marker_df, rel_range in zip(marker_dfs, rel_ranges)]
            cut_off_dfs["UNPOWERED"][t_unpowered]["MOTOR_DATA"] = [motor_df.loc[rel_range.min:rel_range.max] for motor_df, rel_range in zip(motor_dfs, rel_ranges)]
        
        subject_data[subject][session]["cut_off_dfs"] = cut_off_dfs

### Convert index to percentage

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        cut_off_dfs = subject_data[subject][session]["cut_off_dfs"]

        for profile in cut_off_dfs["ASSISTED"].keys():
            emg, motor, marker = cut_off_dfs["ASSISTED"][profile].values()

            for i in range(len(emg)):
                emg[i].reset_index(inplace=True)
                motor[i].reset_index(inplace=True)
                marker[i].reset_index(inplace=True)

                emg[i].index = (emg[i].index - emg[i].index.min())/(emg[i].index.max() - emg[i].index.min()) * 100
                motor[i].index = (motor[i].index - motor[i].index.min())/(motor[i].index.max() - motor[i].index.min()) * 100
                marker[i].index = (marker[i].index - marker[i].index.min())/(marker[i].index.max() - marker[i].index.min()) * 100

                emg[i].set_index(emg[i].index, inplace=True)
                motor[i].set_index(motor[i].index, inplace=True)
                marker[i].set_index(marker[i].index, inplace=True)

                emg[i].index.name = "Percentage"
                motor[i].index.name = "Percentage"
                marker[i].index.name = "Percentage"

        for unpowered_moment in cut_off_dfs['UNPOWERED'].keys():
            emg, motor, marker = cut_off_dfs['UNPOWERED'][unpowered_moment].values()
            
            for i in range(len(emg)):
                emg[i].reset_index(inplace=True)
                motor[i].reset_index(inplace=True)
                marker[i].reset_index(inplace=True)
                
                emg[i].index = (emg[i].index - emg[i].index.min())/(emg[i].index.max() - emg[i].index.min()) * 100
                motor[i].index = (motor[i].index - motor[i].index.min())/(motor[i].index.max() - motor[i].index.min()) * 100
                marker[i].index = (marker[i].index - marker[i].index.min())/(marker[i].index.max() - marker[i].index.min()) * 100


                emg[i].set_index(emg[i].index, inplace=True)
                motor[i].set_index(motor[i].index, inplace=True)
                marker[i].set_index(marker[i].index, inplace=True)

                emg[i].index.name = "Percentage"
                motor[i].index.name = "Percentage"
                marker[i].index.name = "Percentage"

### Adjust range such that the motor peak is aligned with %

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        relevant_ranges = subject_data[subject][session]["relevant_ranges"]
        cut_off_dfs = subject_data[subject][session]["cut_off_dfs"]
                
        shift_motor_data = deepcopy(relevant_ranges)
        profile_peaks = {profile: profile.split("_")[1][:-1] for profile in cut_off_dfs["ASSISTED"].keys()}

        for profile in cut_off_dfs["ASSISTED"].keys():
            for i, motor_df in enumerate(cut_off_dfs["ASSISTED"][profile]["MOTOR_DATA"]):
                old_range = relevant_ranges[profile]["ranges"][i]

                max_ind = motor_df.F_Y.argmax()
                old_peak_time = motor_df.iloc[max_ind].time

                closest_index_value = abs(motor_df.index - profile_infos[profile]["peak_time"]).argmin()
                closest_t_ind = motor_df.index[closest_index_value]
                new_peak_time = motor_df.loc[closest_t_ind].time

                shift_by = new_peak_time - old_peak_time

                new_range = time_range(old_range.min - shift_by, old_range.max - shift_by)
                shift_motor_data[profile]["ranges"][i] = new_range

        subject_data[subject][session]["shifted_motor_ranges"] = shift_motor_data

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        filtered_session_data = subject_data[subject][session]["filtered_session_data"]
        relevant_ranges = subject_data[subject][session]["relevant_ranges"]
        shifted_motor_ranges = subject_data[subject][session]["shifted_motor_ranges"]

        cut_off_dfs = {"ASSISTED": dict(), "UNPOWERED": dict()}

        for profile in filtered_session_data["ASSISTED"].keys():
            rel_ranges = relevant_ranges[profile]["ranges"] # Get first iteration range
            motor_ranges = shifted_motor_ranges[profile]["ranges"]
            cut_off_dfs["ASSISTED"][profile] = deepcopy(data_dict)

            emg_dfs, motor_dfs, marker_dfs = filtered_session_data["ASSISTED"][profile].values()

            cut_off_dfs["ASSISTED"][profile]["EMG"] = [emg_df.loc[rel_range.min:rel_range.max] for emg_df, rel_range in zip(emg_dfs, rel_ranges)]
            cut_off_dfs["ASSISTED"][profile]["MOTOR_DATA"] = [motor_df.loc[rel_range.min:rel_range.max] for motor_df, rel_range in zip(motor_dfs, motor_ranges)]
            cut_off_dfs["ASSISTED"][profile]["MARKER_DATA"] = [marker_df.loc[rel_range.min:rel_range.max] for marker_df, rel_range in zip(marker_dfs, rel_ranges)]

            for motor_df, rel_range in zip(motor_dfs, rel_ranges):
                if rel_range.min > motor_df.index[-1]:
                    print("Warning: cut-off range starts after motor data. Keeping whole range")
                    rel_range = rel_range._replace(min=motor_df.index[0])
                # cut_off_dfs["ASSISTED"][profile]["MOTOR_DATA"].append(motor_df.loc[rel_range.min:rel_range.max])

        for t_unpowered in filtered_session_data["UNPOWERED"].keys():
            rel_ranges = relevant_ranges[f"UNPOWERED_{t_unpowered}"]["ranges"] # Get first iteration range

            cut_off_dfs["UNPOWERED"][t_unpowered] = deepcopy(data_dict)

            emg_dfs, motor_dfs, marker_dfs = filtered_session_data["UNPOWERED"][t_unpowered].values()

            cut_off_dfs["UNPOWERED"][t_unpowered]["EMG"] = [emg_df.loc[rel_range.min:rel_range.max] for emg_df, rel_range in zip(emg_dfs, rel_ranges)]
            cut_off_dfs["UNPOWERED"][t_unpowered]["MARKER_DATA"] = [marker_df.loc[rel_range.min:rel_range.max] for marker_df, rel_range in zip(marker_dfs, rel_ranges)]
            cut_off_dfs["UNPOWERED"][t_unpowered]["MOTOR_DATA"] = [motor_df.loc[rel_range.min:rel_range.max] for motor_df, rel_range in zip(motor_dfs, rel_ranges)]
        
        subject_data[subject][session]["cut_off_dfs"] = cut_off_dfs

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        cut_off_dfs = subject_data[subject][session]["cut_off_dfs"]

        for profile in cut_off_dfs["ASSISTED"].keys():
            emg, motor, marker = cut_off_dfs["ASSISTED"][profile].values()

            for i in range(len(emg)):
                emg[i].reset_index(inplace=True)
                motor[i].reset_index(inplace=True)
                marker[i].reset_index(inplace=True)

                emg[i].index = (emg[i].index - emg[i].index.min())/(emg[i].index.max() - emg[i].index.min()) * 100
                motor[i].index = (motor[i].index - motor[i].index.min())/(motor[i].index.max() - motor[i].index.min()) * 100
                marker[i].index = (marker[i].index - marker[i].index.min())/(marker[i].index.max() - marker[i].index.min()) * 100

                emg[i].set_index(emg[i].index, inplace=True)
                motor[i].set_index(motor[i].index, inplace=True)
                marker[i].set_index(marker[i].index, inplace=True)

                emg[i].index.name = "Percentage"
                motor[i].index.name = "Percentage"
                marker[i].index.name = "Percentage"

        for unpowered_moment in cut_off_dfs['UNPOWERED'].keys():
            emg, motor, marker = cut_off_dfs['UNPOWERED'][unpowered_moment].values()
            
            for i in range(len(emg)):
                emg[i].reset_index(inplace=True)
                motor[i].reset_index(inplace=True)
                marker[i].reset_index(inplace=True)
                
                emg[i].index = (emg[i].index - emg[i].index.min())/(emg[i].index.max() - emg[i].index.min()) * 100
                motor[i].index = (motor[i].index - motor[i].index.min())/(motor[i].index.max() - motor[i].index.min()) * 100
                marker[i].index = (marker[i].index - marker[i].index.min())/(marker[i].index.max() - marker[i].index.min()) * 100


                emg[i].set_index(emg[i].index, inplace=True)
                motor[i].set_index(motor[i].index, inplace=True)
                marker[i].set_index(marker[i].index, inplace=True)

                emg[i].index.name = "Percentage"
                motor[i].index.name = "Percentage"
                marker[i].index.name = "Percentage"

In [ ]:
subject = "MIH02"
session = "December_20"

cut_off_dfs = subject_data[subject][session]["cut_off_dfs"]
unique_muscles = get_unique_muscles(subject_data[subject][session]["filtered_session_data"])

profiles = cut_off_dfs["ASSISTED"].keys()

n_rows = len(cut_off_dfs["ASSISTED"]["peak_77%_62N"]["MOTOR_DATA"])
n_cols = len(profiles)

target_muscle = "RF"

fig, axs = plt.subplots(n_rows, n_cols, figsize=(30, 15))  # Adjust as needed
fig.suptitle(f"{subject}\nAssisted activations for {target_muscle}\n for each iteration of each profile", fontsize=16)

for col, profile in enumerate(profiles):
    for row in range(len(cut_off_dfs["ASSISTED"][profile]["MARKER_DATA"])):
        motor_df = cut_off_dfs["ASSISTED"][profile]["MOTOR_DATA"][row]  # Adjusted for demonstration
        emg_df = cut_off_dfs["ASSISTED"][profile]["EMG"][row]  # Adjusted for demonstration
        cur_range = relevant_ranges[profile]["ranges"][row]  # Assuming relevant_ranges is structured similarly
        hip_vel = relevant_ranges[profile]["hip_vel"][row]  # Assuming relevant_ranges is structured similarly

        axs[row, col].plot(motor_df.index, motor_df.F_Y, label="F_Y", color="g")
        axs[row, col].set_ylim(0, 70)
        axs[row, col].tick_params(axis='y', labelcolor='g')

        ax2 = axs[row, col].twinx()
        ax2.plot(emg_df.index, emg_df[f"{target_muscle}_LEFT"], label="LEFT")
        ax2.plot(emg_df.index, emg_df[f"{target_muscle}_RIGHT"], label="RIGHT")
        ax2.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
        ax2.tick_params(axis='y', labelsize=16)

        if col == len(profiles) - 1:
            ax2.set_ylabel("EMG (% MVIC)", fontsize=16)
        ax2.set_ylim(0, 1)  # Set EMG activation scale from 0 to 1

        handles1, labels1 = axs[row, col].get_legend_handles_labels()
        handles2, labels2 = ax2.get_legend_handles_labels()
        handles = handles1 + handles2
        labels = labels1 + labels2

        axs[row, col].legend(handles, labels, loc="upper right", fontsize=10)

        axs[row, col].tick_params(axis='x', labelsize=16)
        axs[row, col].tick_params(axis='y', labelsize=16)
        axs[row, col].xaxis.set_major_formatter(mticker.PercentFormatter())

        axs[row, 0].set_ylabel(f"Iteration {row+1}\nForce (N)\n{target_muscle}", fontsize=16, color='g')
        axs[-1, col].set_xlabel("STS (%)", fontsize=16)

        axs[0, col].set_title(f"{profile}", fontsize=16)

plt.tight_layout(rect=[0, 0.03, 1, 0.98])
plt.show()


### Average activation signals across trials

#### Interpolate data to have same time range

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        cut_off_dfs = subject_data[subject][session]["cut_off_dfs"]        

        interpolated_dfs = deepcopy(cut_off_dfs)

        for profile in cut_off_dfs["ASSISTED"].keys():
            emg, motor, marker = cut_off_dfs["ASSISTED"][profile].values()

            emg_len = min([len(emg_df) for emg_df in emg])
            motor_len = min([len(motor_df) for motor_df in motor])
            marker_len = min([len(marker_df) for marker_df in marker])

            for i in range(len(emg)):
                interpolated_dfs["ASSISTED"][profile]["EMG"][i] = interpolate_dataframe_to_length(emg[i], emg_len)
                interpolated_dfs["ASSISTED"][profile]["MOTOR_DATA"][i] = interpolate_dataframe_to_length(motor[i], motor_len)
                interpolated_dfs["ASSISTED"][profile]["MARKER_DATA"][i] = interpolate_dataframe_to_length(marker[i], marker_len)
                
        for unpowered_time in cut_off_dfs["UNPOWERED"].keys():
            emg, motor, marker = cut_off_dfs["UNPOWERED"][unpowered_time].values()

            emg_len = min([len(emg_df) for emg_df in emg])
            motor_len = min([len(motor_df) for motor_df in motor])
            marker_len = min([len(marker_df) for marker_df in marker])

            for i in range(len(emg)):
                interpolated_dfs["UNPOWERED"][unpowered_time]["EMG"][i] = interpolate_dataframe_to_length(emg[i], emg_len)
                interpolated_dfs["UNPOWERED"][unpowered_time]["MOTOR_DATA"][i] = interpolate_dataframe_to_length(motor[i], motor_len)
                interpolated_dfs["UNPOWERED"][unpowered_time]["MARKER_DATA"][i] = interpolate_dataframe_to_length(marker[i], marker_len)
        
        subject_data[subject][session]["interpolated_dfs"] = interpolated_dfs

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        cut_off_dfs = subject_data[subject][session]["cut_off_dfs"]

        interpolated_dfs = deepcopy(cut_off_dfs)

        # Initialize variables to store the minimum lengths for each type of data
        min_emg_len = float('inf')
        min_motor_len = float('inf')
        min_marker_len = float('inf')

        # Step 1: Identify the Minimum Lengths across all "ASSISTED" profiles
        for profile in cut_off_dfs["ASSISTED"].keys():
            emg, motor, marker = cut_off_dfs["ASSISTED"][profile].values()

            min_emg_len = min(min_emg_len, min([len(emg_df) for emg_df in emg]))
            min_motor_len = min(min_motor_len, min([len(motor_df) for motor_df in motor]))
            min_marker_len = min(min_marker_len, min([len(marker_df) for marker_df in marker]))

        # Step 2: Apply Interpolation to "ASSISTED"
        for profile in cut_off_dfs["ASSISTED"].keys():
            emg, motor, marker = cut_off_dfs["ASSISTED"][profile].values()

            for i in range(len(emg)):
                interpolated_dfs["ASSISTED"][profile]["EMG"][i] = interpolate_dataframe_to_length(emg[i], min_emg_len)
            for i in range(len(motor)):
                interpolated_dfs["ASSISTED"][profile]["MOTOR_DATA"][i] = interpolate_dataframe_to_length(motor[i], min_motor_len)
            for i in range(len(marker)):
                interpolated_dfs["ASSISTED"][profile]["MARKER_DATA"][i] = interpolate_dataframe_to_length(marker[i], min_marker_len)

        # Step 3: Apply the Same Interpolation to "UNPOWERED"
        for unpowered_time in cut_off_dfs["UNPOWERED"].keys():
            emg, motor, marker = cut_off_dfs["UNPOWERED"][unpowered_time].values()

            for i in range(len(emg)):
                interpolated_dfs["UNPOWERED"][unpowered_time]["EMG"][i] = interpolate_dataframe_to_length(emg[i], min_emg_len)
            for i in range(len(motor)):
                interpolated_dfs["UNPOWERED"][unpowered_time]["MOTOR_DATA"][i] = interpolate_dataframe_to_length(motor[i], min_motor_len)
            for i in range(len(marker)):
                interpolated_dfs["UNPOWERED"][unpowered_time]["MARKER_DATA"][i] = interpolate_dataframe_to_length(marker[i], min_marker_len)
        
        subject_data[subject][session]["interpolated_dfs"] = interpolated_dfs

### Average activations and motor data

In [ ]:
# Drop time column
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        interpolated_dfs = subject_data[subject][session]["interpolated_dfs"]
        for profile in interpolated_dfs["ASSISTED"].keys():
            for emg_df, motor_df, marker_df in zip(*interpolated_dfs["ASSISTED"][profile].values()):
                emg_df.drop(columns="TIME", inplace=True, errors='ignore')
                marker_df.drop(columns="TIME", inplace=True, errors='ignore')

        for unpowered_time in interpolated_dfs["UNPOWERED"].keys():
            for emg_df, motor_df, marker_df in zip(*interpolated_dfs["UNPOWERED"][unpowered_time].values()):
                emg_df.drop(columns="TIME", inplace=True, errors='ignore')
                marker_df.drop(columns="TIME", inplace=True, errors='ignore')

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        interpolated_dfs = subject_data[subject][session]["interpolated_dfs"]
        filtered_session_data = subject_data[subject][session]["filtered_session_data"]
                
        averaged_dfs = {"ASSISTED": {}, "UNPOWERED": {}}
        muscles = interpolated_dfs["UNPOWERED"]["BEFORE"]["EMG"][0].columns

        motor_assisted_columns = interpolated_dfs["ASSISTED"][list(interpolated_dfs["ASSISTED"].keys())[0]]["MOTOR_DATA"][0].columns
        motor_unpowered_columns = interpolated_dfs["UNPOWERED"]["BEFORE"]["MOTOR_DATA"][0].columns

        for mode, columns in zip(["ASSISTED", "UNPOWERED"], [motor_assisted_columns, motor_unpowered_columns]):
            for profile in interpolated_dfs[mode].keys():
                averaged_dfs[mode][profile] = {"EMG": {"mean": None, "std": None}, "MOTOR_DATA": {"mean": None, "std": None}}

                # EMG Data
                muscles_means = {}
                muscles_std = {}

                for emg in interpolated_dfs[mode][profile]["EMG"]:
                    for muscle in emg.columns:
                        if muscle not in muscles_means:
                            muscles_means[muscle] = []
                            muscles_std[muscle] = []
                        muscles_means[muscle].append(emg[muscle])
                        muscles_std[muscle].append(emg[muscle])

                for muscle, dfs in muscles_means.items():
                    merged = pd.concat(dfs, axis=1)
                    muscles_means[muscle] = merged.mean(axis=1)
                    muscles_std[muscle] = merged.std(axis=1)

                averaged_dfs[mode][profile]["EMG"]["mean"] = smooth_dataframe(pd.DataFrame.from_dict(muscles_means), window_size=1)
                averaged_dfs[mode][profile]["EMG"]["std"] = smooth_dataframe(pd.DataFrame.from_dict(muscles_std), window_size=1)

                # MOTOR Data
                motor_means = {}
                motor_std = {}

                for motor_data in interpolated_dfs[mode][profile]["MOTOR_DATA"]:
                    for column in columns:
                        if column not in motor_means:
                            motor_means[column] = []
                            motor_std[column] = []
                        motor_means[column].append(motor_data[column])
                        motor_std[column].append(motor_data[column])

                for column, dfs in motor_means.items():
                    merged = pd.concat(dfs, axis=1)
                    motor_means[column] = merged.mean(axis=1)
                    motor_std[column] = merged.std(axis=1)

                averaged_dfs[mode][profile]["MOTOR_DATA"]["mean"] = smooth_dataframe(pd.DataFrame.from_dict(motor_means))
                averaged_dfs[mode][profile]["MOTOR_DATA"]["std"] = smooth_dataframe(pd.DataFrame.from_dict(motor_std))

        averaged_dfs["UNPOWERED"]["BASELINE"] = {"EMG": {"mean": None, "std": None}, "MOTOR_DATA": {"mean": None, "std": None}}

        if "AFTER" in filtered_session_data["UNPOWERED"].keys():
            averaged_dfs["UNPOWERED"]["BASELINE"]["EMG"]["mean"] = (averaged_dfs["UNPOWERED"]["BEFORE"]["EMG"]["mean"] + averaged_dfs["UNPOWERED"]["AFTER"]["EMG"]["mean"]) / 2
            averaged_dfs["UNPOWERED"]["BASELINE"]["EMG"]["std"] = (averaged_dfs["UNPOWERED"]["BEFORE"]["EMG"]["std"] + averaged_dfs["UNPOWERED"]["AFTER"]["EMG"]["std"]) / 2
        else:
            averaged_dfs["UNPOWERED"]["BASELINE"]["EMG"]["mean"] = averaged_dfs["UNPOWERED"]["BEFORE"]["EMG"]["mean"]
            averaged_dfs["UNPOWERED"]["BASELINE"]["EMG"]["std"] = averaged_dfs["UNPOWERED"]["BEFORE"]["EMG"]["std"]
        
        subject_data[subject][session]["averaged_dfs"] = averaged_dfs

In [ ]:
enable_paper_scaling = False

# Based on target muscles
muscle_values = {
    "RF": 0.46,
    "BF": 0.16,
    "VM": 0.9
}

activation_scaling_factor = {
    "RF_RIGHT": None,
    "RF_LEFT": None,
    "BF_RIGHT": None,
    "BF_LEFT": None,
    "VM_RIGHT": None,
    "VM_LEFT": None
}
if enable_paper_scaling:
    for muscle_data in averaged_dfs["UNPOWERED"]["BEFORE"]["EMG"].values():
        for muscle in muscle_data.columns:
            if "RF" in muscle:
                activation_scaling_factor[muscle] = muscle_data[muscle].max() / muscle_values["RF"]
                muscle_data[muscle] /= activation_scaling_factor[muscle] 
            elif "BF" in muscle:
                activation_scaling_factor[muscle] = muscle_data[muscle].max() / muscle_values["BF"]
                muscle_data[muscle] /= activation_scaling_factor[muscle] 
            elif "VM" in muscle:
                activation_scaling_factor[muscle] = muscle_data[muscle].max() / muscle_values["VM"]
                muscle_data[muscle] /= activation_scaling_factor[muscle] 

    for profile in averaged_dfs["ASSISTED"].keys():
        for muscle_data in averaged_dfs["ASSISTED"][profile]["EMG"].values():
            for muscle in muscle_data.columns:
                muscle_data[muscle] /= activation_scaling_factor[muscle]
    # for profile in filtered_session_data["ASSISTED"].keys():
    #     for i, filtered_emg in enumerate(filtered_session_data["ASSISTED"][profile]["EMG"]):
    #         filtered_emg /= mvic_df.iloc[0]

In [ ]:
subject = "MIH02"
session = "December_20"

averaged_dfs = subject_data[subject][session]["averaged_dfs"]
unique_muscles = get_unique_muscles(subject_data[subject][session]["session_data"])

n_muscles = len(averaged_dfs["ASSISTED"]["peak_27%_62N"]["EMG"]["mean"].columns)

fig, ax = plt.subplots(nrows=n_muscles+1, ncols=n_prof, figsize=(30, 20))

if enable_paper_scaling:
    fig.suptitle(f"{subject}\nEMG activations (paper match scaling)", fontsize=30)
else:
    fig.suptitle(f"{subject}\nEMG activations (MVIC scaled)", fontsize=30)

for i, profile in enumerate(averaged_dfs["ASSISTED"].keys()):
    force_data = averaged_dfs["ASSISTED"][profile]["MOTOR_DATA"]

    
    closest_index_value = abs(force_data["mean"].index - profile_infos[profile]["peak_time"]).argmin()
    closest_t_ind = force_data["mean"].index[closest_index_value]

    ax[0, i].set_title(profile, fontsize=20)
    ax[0, i].plot(force_data["mean"].index, force_data["mean"].F_Y, label=r"$F_Y$", color='green')
    ax[0, i].axvline(closest_t_ind, color="red")
    ax[0, i].fill_between(force_data["mean"].index, force_data["mean"].F_Y - force_data["std"].F_Y, force_data["mean"].F_Y + force_data["std"].F_Y, alpha=0.2, color='green')
    ax[0, 0].set_ylabel('Force (N)', color='green', fontsize=16)
    ax[0, i].tick_params(axis='y', labelcolor='green', labelsize=16)
    ax[0, i].yaxis.set_major_formatter("{x:.0f}N")

    ax2 = ax[0, i].twinx()
    ax2.plot(force_data["mean"].index, force_data["mean"].Percentage/100, label="STS %", linestyle="--", color="purple")
    if i ==len(averaged_dfs["ASSISTED"].keys())-1:
        ax2.set_ylabel('Measured STS %', color='purple', fontsize=16)
    ax2.tick_params(axis='y', labelcolor='purple', labelsize=16)
    ax2.xaxis.set_major_formatter(mticker.PercentFormatter())
    ax2.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
    ax2.set_ylim(0, 1)

    for j in range(1, n_muscles+1):
        muscle = averaged_dfs["ASSISTED"][profile]["EMG"]["mean"].columns[j-1]
        peak_time = profile_infos[profile]["peak_time"]
        
        if i == 0:
            ax[j, i].set_ylabel(f"{muscle}\nEMG (% of MVIC)", fontsize=16)


        ax[-1, i].set_xlabel("Percentage of STS", fontsize=16)
        ax[j, i].xaxis.set_major_formatter(mticker.PercentFormatter())
        
        emg_mean = averaged_dfs["ASSISTED"][profile]["EMG"]["mean"][muscle]
        emg_std = averaged_dfs["ASSISTED"][profile]["EMG"]["std"][muscle]

        closest_peak_index = np.abs(emg_mean.index.astype(float) - peak_time).argmin()

        emg_baseline_mean = averaged_dfs["UNPOWERED"]["BASELINE"]["EMG"]["mean"][muscle]
        emg_baseline_std = averaged_dfs["UNPOWERED"]["BASELINE"]["EMG"]["std"][muscle]
           

        ax[j, i].plot(emg_mean.index, emg_mean, label=f"Trials")
        ax[j, i].fill_between(emg_mean.index, emg_mean - emg_std, emg_mean + emg_std, alpha=0.2)
        ax[j, i].plot(emg_baseline_mean.index, emg_baseline_mean, label=f"Baseline")
        ax[j, i].fill_between(emg_baseline_mean.index, emg_baseline_mean - emg_baseline_std, emg_baseline_mean + emg_baseline_std, alpha=0.2)
        ax[j, i].tick_params(axis='x', labelsize=16)
        ax[j, i].tick_params(axis='y', labelsize=16)
        ax[j, i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
        
        force_handles, force_labels = ax[0, i].get_legend_handles_labels()

        handles1, labels1 = ax[j, i].get_legend_handles_labels()
        handles2, labels2 = ax2.get_legend_handles_labels()
        handles = handles1 + handles2 + force_handles
        
        labels = labels1 + labels2 + force_labels
    
legend = fig.legend(handles, labels, loc="upper center", ncols=len(labels), fontsize=20, bbox_to_anchor=(0.5, 0.93))

for line in legend.get_lines():
    line.set_linewidth(4)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

### Calculate mean activation delta

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        averaged_dfs = subject_data[subject][session]["averaged_dfs"]      

        mean_profile_activations = deepcopy(averaged_dfs)

        for profile in averaged_dfs["ASSISTED"].keys():
            mean_profile_activations["ASSISTED"][profile]["EMG"]["mean"] = averaged_dfs["ASSISTED"][profile]["EMG"]["mean"].mean()
            mean_profile_activations["ASSISTED"][profile]["EMG"]["std"] = averaged_dfs["ASSISTED"][profile]["EMG"]["std"].mean()


        mean_profile_activations["UNPOWERED"]["BASELINE"]["EMG"]["mean"] = averaged_dfs["UNPOWERED"]["BASELINE"]["EMG"]["mean"].mean()
        mean_profile_activations["UNPOWERED"]["BASELINE"]["EMG"]["std"] = averaged_dfs["UNPOWERED"]["BASELINE"]["EMG"]["std"].mean()

        subject_data[subject][session]["mean_profile_activations"] = mean_profile_activations

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        mean_profile_activations = subject_data[subject][session]["mean_profile_activations"]

        n_profiles = len(mean_profile_activations["ASSISTED"].keys())
        muscles = mean_profile_activations["UNPOWERED"]["BASELINE"]["EMG"]["mean"].index
        
        muscle_deltas = []  # For storing deltas
        muscle_activations = []  # For storing activations

        for profile in profiles:
            muscle_means_delta = []  # To calculate deltas
            muscle_stds_delta = []  # To calculate deltas
            muscle_means_activation = []  # To store activations
            muscle_stds_activation = []  # To store activations

            for muscle in muscles:
                mean_assisted = mean_profile_activations["ASSISTED"][profile]["EMG"]["mean"][muscle]
                std_assisted = mean_profile_activations["ASSISTED"][profile]["EMG"]["std"][muscle]
                
                mean_baseline = mean_profile_activations["UNPOWERED"]["BASELINE"]["EMG"]["mean"][muscle]
                std_baseline = mean_profile_activations["UNPOWERED"]["BASELINE"]["EMG"]["std"][muscle]

                # Calculate deltas
                mean_delta = (mean_assisted - mean_baseline) / mean_baseline
                std_delta = (std_assisted + std_baseline)  # Simplified for demonstration; adjust as necessary
                
                # Append delta calculations
                muscle_means_delta.append(mean_delta)
                muscle_stds_delta.append(std_delta)
                muscle_deltas.append({"Profile": profile, "Muscle": muscle, "Mean Delta": mean_delta, "STD Delta": std_delta})
                
                # Append activations
                muscle_means_activation.append(mean_assisted)
                muscle_stds_activation.append(std_assisted)
                muscle_activations.append({"Profile": profile, "Muscle": muscle, "Mean Activation": mean_assisted, "STD Activation": std_assisted})
                        
                
            # Append average of all muscles to each profile for deltas
            muscles_delta_df = pd.DataFrame({"means": muscle_means_delta, "stds": muscle_stds_delta}, index=muscles).mean()
            muscle_deltas.append({"Profile": profile, "Muscle": "AVERAGE", "Mean Delta": muscles_delta_df.means, "STD Delta": muscles_delta_df.stds})
            
            # Append average of all muscles to each profile for activations
            muscles_activation_df = pd.DataFrame({"means": muscle_means_activation, "stds": muscle_stds_activation}, index=muscles).mean()
            muscle_activations.append({"Profile": profile, "Muscle": "AVERAGE", "Mean Activation": muscles_activation_df.means, "STD Activation": muscles_activation_df.stds})
            
        # Convert the collected data into DataFrames and store them under their respective keys
        df_deltas = pd.DataFrame(muscle_deltas)
        df_activations = pd.DataFrame(muscle_activations)
        subject_data[subject][session]["mean_delta_activations"] = df_deltas
        subject_data[subject][session]["mean_activations"] = df_activations

In [ ]:
profile_names_paper = []

for name in profile_infos.keys():
    peak_time = name.split("_")[1]
    profile_names_paper.append(f"{peak_time} Peak Timing")
peak_times = [profile['peak_time'] for profile in profile_infos.values()]

In [ ]:
fig, axs = plt.subplots(figsize=(10, 5))
fig.suptitle(f"Mean muscle activation $\Delta$\nper subject and profile", fontsize=20, y=1.02)

profile_deltas = {}

for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        mean_delta_activations = subject_data[subject][session]["mean_delta_activations"]
        muscle_averages = mean_delta_activations[mean_delta_activations["Muscle"] == "AVERAGE"]
        
        muscle_averages.loc[:, "Profile"] = peak_times

        axs.plot(muscle_averages["Profile"], muscle_averages["Mean Delta"], label=subject, marker='*', linewidth=3, markersize=10)

        for index, row in muscle_averages.iterrows():
            if row["Profile"] not in profile_deltas:
                profile_deltas[row["Profile"]] = []
            profile_deltas[row["Profile"]].append(row["Mean Delta"])

average_deltas = {profile: np.mean(deltas) for profile, deltas in profile_deltas.items()}
average_values = list(average_deltas.values())

axs.grid(False)
axs.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
axs.hlines(0, profile_infos['peak_27%_62N']['peak_time'], profile_infos['peak_77%_62N']['peak_time'], color='black', linestyle='--')
axs.set_xlabel("Peak Timing of Assistance (%)")
axs.set_xticks(peak_times)
axs.set_ylabel("Muscle Activity\nReduction (%)")

axs.spines['top'].set_visible(False)
axs.spines['right'].set_visible(False)
# axs.spines['bottom'].set_visible(False)  # Adjust thickness of the arrow

# Make x-axis and y-axis arrows
axs.spines['left'].set_linewidth(2)
axs.spines['bottom'].set_linewidth(2)  # Adjust thickness of the arrow
axs.spines['bottom'].set_capstyle('projecting')  # Arrow style
offset = 0.03


axs.set_ylim(-0.4, 0.2)
axs.legend(fontsize=16, title="Subjects", title_fontsize='16', loc='upper right')


plt.savefig("../paper_figures/mean_delta_activations.svg", format='svg')
plt.show()



In [ ]:
# Plot average muscle activations per profile as a line plot
n_subjects = len(subject_data.keys())

fig, axs = plt.subplots(1, n_subjects, figsize=(20, 5))

for i, subject in enumerate(subject_data.keys()):
    for session in subject_data[subject].keys():
        mean_activations = subject_data[subject][session]["mean_activations"]
        for muscle in mean_activations.Muscle.unique():
            muscle_averages = mean_activations[mean_activations["Muscle"] == muscle]

            axs[i].plot(muscle_averages["Profile"], muscle_averages["Mean Activation"], label=muscle, marker='*', linewidth=3, markersize=10)
        axs[i].set_title(f"Subject {subject}", fontsize=20)
        axs[i].legend(loc="upper right", fontsize=10)
        axs[i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
        axs[i].tick_params(rotation=45)
        axs[i].set_xlabel("Profile")
        axs[i].set_ylabel("Mean Activation\n(% of MVIC)")



In [ ]:
n_subjects = len(subject_data.keys())

fig, axs = plt.subplots(1, n_subjects, figsize=(20, 5), squeeze=False)  # Ensure axs is 2D

# Predefined colors for specific muscles
colors = {'BASELINE': '#1f77b4', 'AVERAGE': 'orange'}

for i, subject in enumerate(subject_data.keys()):
    muscle_colors = {}  # Dictionary to dynamically assign colors to each muscle

    for session in subject_data[subject].keys():
        mean_activations = subject_data[subject][session]["mean_activations"]
        muscles = sorted(mean_activations.Muscle.unique())
        
        # Dynamically generate grey shades based on the number of muscles
        grey_count = len([m for m in muscles if m not in colors])
        grey_values = np.linspace(0.8, 0.2, grey_count)  # Light (0.8) to dark (0.2)
        grey_counter = 0
        # Assign grey shades to muscles not in the predefined colors
        for index, muscle in enumerate(muscles):
            if muscle not in colors:
                # Use the grey_counter to access grey_values
                grey = grey_values[grey_counter]
                # Convert grey value to RGB format
                rgb_grey = (grey, grey, grey)
                muscle_colors[muscle] = rgb_grey
                # Increment the grey_counter
                grey_counter += 1
        
        # Update the colors dictionary with dynamically generated grey shades
        colors.update(muscle_colors)
        
        for muscle in muscles:
            muscle_averages = mean_activations[mean_activations["Muscle"] == muscle]
            color = colors.get(muscle)
            
            # Plot line with dynamically assigned color
            axs[0, i].plot(muscle_averages["Profile"], muscle_averages["Mean Activation"], 
                           label=muscle, marker='*', linewidth=3, markersize=10, color=color)
            
            # Place muscle name next to the line on the right
            if not muscle_averages.empty:
                axs[0, i].text(muscle_averages["Profile"].iloc[-1], muscle_averages["Mean Activation"].iloc[-1], 
                               f"  {muscle}", verticalalignment='center', color=color)

        axs[0, i].set_title(f"Subject {subject}", fontsize=20)
        axs[0, i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
        axs[0, i].tick_params(axis='x', rotation=45)
        axs[0, i].set_xlabel("Profile")
        axs[0, i].set_ylabel("Mean Activation\n(% of MVIC)")

# Remove the legend since muscle names are annotated on the plot
for ax in axs.flat:
    ax.legend().set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
all_deltas = []
all_means = []


for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        df_delta = subject_data[subject][session]["mean_delta_activations"]
        df_means = subject_data[subject][session]["mean_activations"]
        df_baseline = subject_data[subject][session]["averaged_dfs"]["UNPOWERED"]["BASELINE"]["EMG"]["mean"]
        
        # Ensuring each DataFrame has 'Subject' and 'Session'
        df_delta['Subject'] = subject
        df_delta['Session'] = session
        df_means['Subject'] = subject
        df_means['Session'] = session
        
        all_deltas.append(df_delta)
        all_means.append(df_means)

df_all_deltas = pd.concat(all_deltas)
df_all_means = pd.concat(all_means)

df_all_deltas.sort_values(by="Muscle", ascending=True, inplace=True)
df_all_means.sort_values(by="Muscle", ascending=True, inplace=True)

In [ ]:
# Create a figure and axis for the plot
fig, ax = plt.subplots(1, n_profiles, figsize=(35, 10), sharey=True)
fig.suptitle("Mean Activation $\Delta$\nMean Activations Across Subjects and Sessions", fontsize=24, y=1.1)


for i, profile in enumerate(sorted(profile_infos.keys())):
    df_deltas_profile = df_all_deltas[df_all_deltas['Profile'] == profile].sort_values(by="Subject")
    df_means_profile = df_all_means[df_all_means['Profile'] == profile].sort_values(by="Subject")

    df_deltas_profile[df_deltas_profile['Muscle'] == "AVERAGE"]

    # Plotting delta activations with consideration for missing data
    sns.barplot(x="Muscle", y="Mean Delta", hue="Subject", data=df_deltas_profile, ax=ax[i], dodge=True)\

    # # Adding error bars for delta activations
    ax[i].set_xlabel("Muscle", fontsize=16)
    ax[i].tick_params(axis='x', labelrotation=45, labelsize=16)
    ax[i].tick_params(axis='y', labelsize=16)
    ax[i].set_ylabel("Mean Activation $\Delta$ (%)", fontsize=20)
    ax[i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))

    handles, labels = ax[i].get_legend_handles_labels()

    ax[i].get_legend().remove()

    
fig.legend(handles, labels, fontsize=20, bbox_to_anchor=(0.525, 1.02))  
plt.tight_layout()
plt.show()

In [ ]:
subject = "MIH02"
session = "December_20"

df_delta = subject_data[subject][session]["mean_delta_activations"]
df_means = subject_data[subject][session]["mean_activations"]

fig, axs = plt.subplots(1, n_profiles, figsize=(30, 10))
fig.suptitle(f"Subject {subject}\nMean muscle activation $\Delta$", fontsize=20, y=1.02)

for i, profile in enumerate(profiles):
    profile_df = df_delta[df_delta["Profile"] == profile]
    sns.barplot(x="Muscle", y="Mean Delta", data=profile_df, ax=axs[i], dodge=False)
    
    # Adding error bars
    for j, muscle in enumerate(profile_df["Muscle"].unique()):
        axs[i].errorbar(j, profile_df[profile_df["Muscle"] == muscle]["Mean Delta"].values[0],
                        yerr=profile_df[profile_df["Muscle"] == muscle]["STD Delta"].values[0], 
                        fmt='none', color='gray', capsize=5, elinewidth=2)
    axs[i].set_title(profile, fontsize=14)
    axs[i].set_xlabel("Muscle", fontsize=16)
    axs[i].set_ylabel("Mean activation delta (%)", fontsize=16)
    axs[i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))

    plt.setp(axs[i].get_xticklabels(), rotation=45, ha="right")

plt.tight_layout(rect=[0, 0, 1, 1])  # Adjust the layout to make room for the suptitle
plt.show()

### Calculate peak activation delta

In [ ]:
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        n_profiles = len(mean_profile_activations["ASSISTED"].keys())
        muscles = mean_profile_activations["UNPOWERED"]["BASELINE"]["EMG"]["mean"].index
        data = []

        for profile in profiles:
            muscle_means = []
            muscle_stds = []

            for muscle in muscles:
                mean_assisted = mean_profile_activations["ASSISTED"][profile]["EMG"]["mean"][muscle]
                std_assisted = mean_profile_activations["ASSISTED"][profile]["EMG"]["std"][muscle]
                
                mean_baseline = mean_profile_activations["UNPOWERED"]["BASELINE"]["EMG"]["mean"][muscle]
                std_baseline = mean_profile_activations["UNPOWERED"]["BASELINE"]["EMG"]["std"][muscle]

                mean_delta = (mean_assisted - mean_baseline) / mean_baseline
                std_delta = (std_assisted + std_baseline) #/ std_baseline

                muscle_means.append(mean_delta)
                muscle_stds.append(std_delta)
                
                data.append({"Profile": profile, "Muscle": muscle, "Mean Delta": mean_delta, "STD Delta": std_delta})
                
            muscles_delta_df = pd.DataFrame({"means": muscle_means, "stds": muscle_stds}, index=muscles).mean()
            data.append({"Profile": profile, "Muscle": "Average", "Mean Delta": muscles_delta_df.means, "STD Delta": muscles_delta_df.stds})
            # Append average of all muscles to each profile
        df = pd.DataFrame(data)

        subject_data[subject][session]["mean_delta_activations"] = df

In [ ]:
# Calculate peak EMG values for each muscle (before and after trials)
for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        cut_off_dfs = subject_data[subject][session]["cut_off_dfs"]
        muscles = subject_data[subject][session]["mean_profile_activations"]["ASSISTED"]["peak_77%_62N"]["EMG"]["mean"].index

        peak_emg_means = []
        
        for t_unpowered in cut_off_dfs["UNPOWERED"].keys():
            peak_means = [df.max() for df in cut_off_dfs["UNPOWERED"][t_unpowered]["EMG"]]
            peak_emg_means.append(sum(peak_means) / len(peak_means))

        peak_emg_means = sum(peak_emg_means) / len(peak_emg_means)

        # Calculate average peak EMG values for each muscle (assisted trials)
        peak_values_assisted = defaultdict(dict)
        for profile in cut_off_dfs["ASSISTED"].keys():
            max_values_assisted = pd.DataFrame([df.max() for df in cut_off_dfs["ASSISTED"][profile]["EMG"]])
            peak_values_assisted[profile]["mean"] = max_values_assisted.mean()
            peak_values_assisted[profile]["std"] = max_values_assisted.std()

        peak_activation_deltas = []

        for profile in profiles:
            peak_means = []
            peak_stds = []

            for muscle in muscles:
                peak_mean = peak_values_assisted[profile]["mean"][muscle]
                peak_std = peak_values_assisted[profile]["std"][muscle]

                peak_mean_baseline = peak_emg_means[muscle]
                peak_std_baseline = peak_emg_means[muscle]

                peak_delta = (peak_mean - peak_mean_baseline) / peak_mean_baseline
                peak_std_delta = (peak_std + peak_std_baseline)

                peak_means.append(peak_delta)
                peak_stds.append(peak_std_delta)

                peak_activation_deltas.append({"Profile": profile, "Muscle": muscle, "Mean Delta": peak_delta, "STD Delta": peak_std_delta})

            peaks_df = pd.DataFrame({"means": peak_means, "stds": peak_stds}, index=muscles).mean()
            peak_activation_deltas.append({"Profile": profile, "Muscle": "Average", "Mean Delta": peaks_df.means, "STD Delta": peaks_df.stds})

        peak_activation_deltas_df = pd.DataFrame(peak_activation_deltas)
        subject_data[subject][session]["peak_activation_deltas"] = peak_activation_deltas_df

In [ ]:
fig, axs = plt.subplots(1, n_profiles, figsize=(30, 10), sharey=True)

fig.suptitle(f"Subject {subject}\n" + r"Peak muscle activation $\Delta$", fontsize=20, y=1.02)

for i, profile in enumerate(profiles):
    profile_df = peak_activation_deltas_df[peak_activation_deltas_df["Profile"] == profile]
    sns.barplot(x="Muscle", y="Mean Delta", data=profile_df, ax=axs[i], dodge=False)

    # Adding error bars
    for j, muscle in enumerate(muscles):
        axs[i].errorbar(j, profile_df[profile_df["Muscle"] == muscle]["Mean Delta"].values[0],
                        yerr=profile_df[profile_df["Muscle"] == muscle]["STD Delta"].values[0], 
                        fmt='none', color='gray', capsize=5, elinewidth=2)
    axs[i].set_title(profile, fontsize=14)
    axs[i].set_xlabel("Muscle", fontsize=16)
    axs[i].set_ylabel("Peak activation delta (% of MVIC)", fontsize=16)
    axs[i].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))

    plt.setp(axs[i].get_xticklabels(), rotation=45, ha="right")
    for k, bar in enumerate(axs[i].patches):
        bar.set_edgecolor('black')

    strip_collections = axs[i].collections


plt.tight_layout(rect=[0, 0, 1, 1])  # Adjust the layout to make room for the suptitle
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(15, 5))
fig.suptitle(f"Average PEAK muscle activation $\Delta$\nper subject and profile", fontsize=20, y=1.02)

# Initialize a dictionary to store Mean Delta values for each profile across subjects
profile_deltas = {}

for subject in subject_data.keys():
    for session in subject_data[subject].keys():
        mean_delta_activations = subject_data[subject][session]["peak_activation_deltas"]
        muscle_averages = mean_delta_activations[mean_delta_activations["Muscle"] == "Average"]

        # Plotting each subject's data
        axs.plot(muscle_averages["Profile"], muscle_averages["Mean Delta"], label=subject, marker='*', linewidth=3, markersize=10)
        
        # Accumulate Mean Delta values for averaging later
        for index, row in muscle_averages.iterrows():
            if row["Profile"] not in profile_deltas:
                profile_deltas[row["Profile"]] = []
            profile_deltas[row["Profile"]].append(row["Mean Delta"])

# Calculate and plot the average across subjects for each profile
average_deltas = {profile: np.mean(deltas) for profile, deltas in profile_deltas.items()}
profiles = list(average_deltas.keys())
average_values = list(average_deltas.values())

axs.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
plt.xticks(rotation=45)
axs.hlines(0, 0, len(profiles)-1, color='black', linestyle='--')
axs.set_xlabel("Profile")
axs.set_ylabel("Mean Delta\n(% of baseline)")
axs.set_ylim(-0.4, 0.4)
axs.legend(fontsize=16, title="Subjects", title_fontsize='16', loc='upper right')

plt.show()

In [ ]:
fig, axs = plt.subplots(1, n_subjects, figsize=(10, 5), squeeze=False)  # Ensure axs is always 2D

for i, subject in enumerate(subject_data.keys()):
    for session in subject_data[subject].keys():
        df = subject_data[subject][session]["mean_activations"]
        assisted_mean = df[df["Muscle"] == "AVERAGE"][["Mean Activation", "STD Activation"]].mean()
        baseline_mean = df[df["Muscle"] == "BASELINE"][["Mean Activation", "STD Activation"]].mean()
        
        # Adjusted sns.barplot to plot horizontally
        sns.barplot(y=["Assisted", "Baseline"], 
                    x=[assisted_mean["Mean Activation"], baseline_mean["Mean Activation"]],
                    orient='h', ax=axs[0, i])  # Note the swapped x and y parameters
        
        # Adjust title, labels, and tick parameters for horizontal orientation
        axs[0, i].set_title(f"Subject {subject}", fontsize=20)
        axs[0, i].set_ylabel("Condition", fontsize=16)  # Now y-axis is "Condition"
        axs[0, i].set_xlabel("Mean Activation\n(% of MVIC)", fontsize=16)  # x-axis shows Mean Activation
        
        axs[0, i].tick_params(axis='y', labelsize=14)  # Adjust y-tick labels for "Condition"
        axs[0, i].tick_params(axis='x', labelsize=14)  # Adjust x-tick labels for Mean Activation
        
        axs[0, i].set_xlim(0, 0.5)  # Adjust x-limits to fit the data
        axs[0, i].xaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))  # Apply percent formatter to x-axis

plt.tight_layout()
plt.show()

### Calculate correlation between applied force profile and simulated profile

In [ ]:
from matplotlib.patches import FancyArrowPatch

subject = "MIH02"
session = "December_20"

data = subject_data[subject][session]

prof_27 = "peak_27%_62N"
prof_47 = "peak_47%_62N"

line_names = ['27% peak timing', '47% peak timing']
n_subjects = len(subject_data)  # Total number of subjects

# Creating 3 rows: one for original profiles, one for correlation plots, and one for sum of correlations bar plots
fig, axs = plt.subplots(figsize=(6, 4), constrained_layout=True)

for profile, name in zip(target_splines, profile_names_paper):
    force_Y = profile.force_Y
    sim_profile_interp = interpolate_dataframe_to_length(sim_profile[["force_Y"]], len(force_Y))
    sim_profile_interp.index = profile.Percentage

    # Plot original profiles
    axs.plot(profile.Percentage, force_Y, label=name, linestyle='-', alpha=0.7, linewidth=3)
    # Remove y-axis ticks and labels
    axs.xaxis.set_major_formatter(mticker.PercentFormatter())

axs.plot(sim_profile_interp.index, sim_profile_interp["force_Y"], label='OpenSim Assistance', linestyle='--', color='black', alpha=0.7, linewidth=3)

axs.grid(False)
axs.spines['top'].set_visible(False)
axs.spines['right'].set_visible(False)

axs.spines['left'].set_visible(False)  # Adjust thickness of the arrow
axs.spines['bottom'].set_visible(False)  # Adjust thickness of the arrow

# Make x-axis and y-axis arrows
axs.spines['left'].set_linewidth(2)
axs.spines['bottom'].set_linewidth(1)  # Adjust thickness of the arrow
axs.spines['bottom'].set_capstyle('projecting')  # Arrow style
offset = 0.03

# Add padding to figure

axs.add_patch(FancyArrowPatch((offset, offset), (1 - offset, offset),
                              transform=axs.transAxes, 
                              color='black',
                              arrowstyle='-|>',
                              mutation_scale=20,
                              linewidth=2))

axs.add_patch(FancyArrowPatch((offset, offset), (offset, 1),
                              transform=axs.transAxes, 
                              color='black',
                              arrowstyle='-|>',
                              mutation_scale=20,
                              linewidth=2))
# add padding to figure
# Adjust ticks
axs.xaxis.set_major_locator(mticker.MaxNLocator(1))  # Show only initial and final values  # Remove y-axis ticks
axs.yaxis.set_major_locator(mticker.MaxNLocator(1))  # Show only initial and final values  # Remove y-axis ticks
# Set labels
axs.set_xlabel("Sit-to-Stand (%)")
axs.set_ylabel("Assistance (N)")

# Customize legend
axs.legend(loc=(0.04, 1), frameon=False, fontsize=8)

plt.savefig("../paper_figures/osim_27_47.svg", format='svg', bbox_inches='tight')
plt.show()

In [ ]:
n_subjects = len(subject_data)  # Total number of subjects
# Creating 3 rows: one for original profiles, one for correlation plots, and one for sum of correlations bar plots
fig, axs = plt.subplots(3, n_subjects, figsize=(10 * n_subjects, 18), constrained_layout=True)
fig.suptitle("Correlation and Sum of Correlations Across Subjects", fontsize=16)

if n_subjects == 1:
    axs = axs[:, np.newaxis]  # Ensure axs is 2D even for a single subplot

for subject_idx, (subject, sessions) in enumerate(subject_data.items(), start=1):
    ax_profile = axs[0, subject_idx - 1]  # Axes for the original profiles
    ax_corr = axs[1, subject_idx - 1]  # Axes for the correlation plot
    ax_sum = axs[2, subject_idx - 1]  # Axes for the sum of correlations bar plot
    session_name = next(iter(sessions))  # Assuming there's only one session per subject
    data = sessions[session_name]

    ax_profile.set_title(f"Subject: {subject}\nSession: {session_name}\nOriginal Profiles", fontsize=14)
    ax_corr.set_title("Correlation Plots", fontsize=14)
    total_correlation = []  # List to store sum of correlations for each profile
    
    for profile in data["averaged_dfs"]["ASSISTED"].keys():
        force_Y = data["averaged_dfs"]["ASSISTED"][profile]["MOTOR_DATA"]["mean"].F_Y
        sim_profile_interp = interpolate_dataframe_to_length(sim_profile[["force_Y"]], len(force_Y))
        sim_profile_interp.index = force_Y.index

        # Plot original profiles
        ax_profile.plot(force_Y.index, force_Y, label=f'{profile} Experimental', linestyle='-', alpha=0.7, linewidth=4)
        ax_profile.set_xlabel("Percentage of STS", fontsize=12)
        ax_profile.set_ylabel("Force", fontsize=12)
        ax_profile.xaxis.set_major_formatter(mticker.PercentFormatter())
        ax_profile.yaxis.set_major_formatter(mticker.StrMethodFormatter('{x:,.0f}N'))
        
        # Plot simulation profile as a dashed black line

        # Calculate correlation
        corr = np.correlate(force_Y, sim_profile_interp["force_Y"], 'same')
        total_correlation.append(np.sum(corr))  # Sum of correlation for the current profile

        # Plot correlation for the current profile on the second row
        ax_corr.plot(force_Y.index, corr, label=profile, linewidth=4)
        ax_corr.set_xlabel("Percentage of STS", fontsize=12)
        ax_corr.xaxis.set_major_formatter(mticker.PercentFormatter())

    ax_profile.plot(sim_profile_interp.index, sim_profile_interp["force_Y"], label='Simulation profile', linestyle='--', color='black', alpha=0.7, linewidth=4)
    ax_profile.legend(loc='upper right', fontsize=8)  # Add legend to differentiate profiles
    ax_corr.legend(loc='best', fontsize=8)  # Add legend to differentiate profiles
    max_corr_index = np.argmax(total_correlation)  # Index of the profile with the highest sum of correlation
    
    # Bar plot for the sum of correlations on the third row
    bars = ax_sum.bar(range(len(total_correlation)), total_correlation, tick_label=list(data["averaged_dfs"]["ASSISTED"].keys()))
    bars[max_corr_index].set_color('red')  # Highlight the max with red color

    ax_sum.set_title("Total Correlation by Profile", fontsize=14)
    ax_sum.set_ylabel("Sum of Correlation", fontsize=12)
    ax_sum.set_xlabel("Profile", fontsize=12)

plt.show()